In [62]:
import pandas as pd 
import numpy as np 
from pmdarima.model_selection import train_test_split
from neuralprophet import NeuralProphet
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error
import mlflow
import mlflow.pyfunc
import os
from datetime import timedelta, datetime
import calendar
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings("ignore")

# Input data 

In [63]:
df = pd.read_csv("train_data.csv")
df

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
91295,2017-12-27,1,50,38
91296,2017-12-28,1,50,52
91297,2017-12-29,1,50,59
91298,2017-12-30,1,50,66


In [64]:
"""data for initial first model traning"""
base_data = df[df["date"] < "2017-01-01"]
base_data

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
90930,2016-12-27,1,50,37
90931,2016-12-28,1,50,44
90932,2016-12-29,1,50,47
90933,2016-12-30,1,50,46


In [65]:
base_data.to_csv(r"D:\MLOPs POC\python files\samp_data\data_2013_to_2016.csv")

In [4]:
"""data for foracsating next month and building new model with updated data"""
des_month_data = df[df["date"] >= "2017-12-01"]
nov_month_data = df[(df["date"] >= "2017-11-01") & (df["date"] < "2017-12-01")]
oct_month_data = df[(df["date"] >= "2017-10-01") & (df["date"] < "2017-11-01")]
sep_month_data = df[(df["date"] >= "2017-09-01") & (df["date"] < "2017-10-01")]
aug_month_data = df[(df["date"] >= "2017-08-01") & (df["date"] < "2017-09-01")]
jul_month_data = df[(df["date"] >= "2017-07-01") & (df["date"] < "2017-08-01")]
jun_month_data = df[(df["date"] >= "2017-06-01") & (df["date"] < "2017-07-01")]
may_month_data = df[(df["date"] >= "2017-05-01") & (df["date"] < "2017-06-01")]
apr_month_data = df[(df["date"] >= "2017-04-01") & (df["date"] < "2017-05-01")]
mar_month_data = df[(df["date"] >= "2017-03-01") & (df["date"] < "2017-04-01")]
feb_month_data = df[(df["date"] >= "2017-02-01") & (df["date"] < "2017-03-01")]
jan_month_data = df[(df["date"] >= "2017-01-01") & (df["date"] < "2017-02-01")]

# Phase 1 : Preprocess

In [5]:
"""In pre process first grouping by items i.e. 50 
   then by droping unique columns geting final X and Y dataframe"""
def pre_pros(df):
    y = {}
    X = {}
    for item,data in df.groupby("item"):
        data = data.drop([col for col in data.columns if data[col].nunique()==1], axis=1)
        data.rename(columns={"sales" : f"sales_{item}"}, inplace=True)
        X.update({"date" : data["date"].values})
        y.update({f"sales_{item}" : data[f"sales_{item}"].values})
    df_X = pd.DataFrame(X)
    df_y = pd.DataFrame(y)
    df_final = pd.concat([df_X, df_y], axis=1)
    
    if not os.path.exists(r"D:\MLOPs POC\prepros_sales_data\updated_sales_data.csv"):
        os.mkdir("prepros_sales_data") 
        df_final.to_csv(r"D:\MLOPs POC\prepros_sales_data\updated_sales_data.csv", index=False )
    else:
        df_old = pd.read_csv(r"D:\MLOPs POC\prepros_sales_data\updated_sales_data.csv")
        df_new = df_final
        df_final = pd.concat([df_old,df_new], axis=0, ignore_index= True)
        df_final.to_csv(r"D:\MLOPs POC\prepros_sales_data\updated_sales_data.csv", index=False)
    return df_final

# Phase 2 :  Build Model

In [6]:
"""model evaluation metrics"""
item_metrics = {}
def eval_model(train_true,train_pred,test_true,test_pred):
    MAPE_train = mean_absolute_percentage_error(train_true,train_pred)
    MAPE_test = mean_absolute_percentage_error(test_true,test_pred)
    dict_eval = {"MAPE_train" : MAPE_train, "MAPE_test" : MAPE_test}
    return dict_eval

"""train test split"""
def train_test_data(data, split_test):
    train, test = train_test_split(data, test_size = split_test) # splite by dates 
    return {"train" : train, "test" : test}

"""building nuralprofet model for each item i.e. 50 models"""
def model_build_NP(data, split_test = 0.2):    
    """using only train and test data"""
    data_dic =  train_test_data(data, split_test)
    train = data_dic["train"]
    test = data_dic["test"]
    # date time formate
    train["date"] = pd.to_datetime(train["date"], format="%Y-%m-%d") 
    test["date"] = pd.to_datetime(test["date"], format="%Y-%m-%d")
    
    model_dict = {}
    col_names = train.columns
    for y_indx in range(1, len(train.columns)):
        """item key for model_dict"""
        item_name = col_names[y_indx]
            
        #with mlflow.start_run(run_name=f"{item_name}_run") as run:
        """train data processing as pere NP formate"""
        tsdf_train = train.iloc[:, [0, y_indx]]
        tsdf_train.columns = ["ds", "y"] 

        """test data processing as pere NP formate"""
        tsdf_test = test.iloc[:, [0, y_indx]]
        tsdf_test.columns = ["ds", "y"] 

        """model traning"""
        # n_lags= 7 perform good but issue in 30 days data creation for fourcast
        npf = Prophet()
        npf.fit(tsdf_train)
        model_dict[item_name] = npf

        """train and test pred and true values"""
        pred_train = npf.predict(tsdf_train)
        pred_test = npf.predict(tsdf_test)
        # change with lag eg. lag 7 : pred.iloc[7:,1], pred.iloc[7:,2]
        train_true = pred_train.iloc[:,1]
        train_pred = pred_train.iloc[:,2]
        test_true = pred_test.iloc[:,1]
        test_pred = pred_test.iloc[:,2]

        """model evaluation metrics"""
        dict_eval = eval_model(train_true,train_pred,test_true,test_pred)
        item_metrics[item_name] = dict_eval

    return model_dict

# phase 3: final model building with all data (we can add model compare in future here)

In [7]:
def final_model_build_NP(data): 
    """considering whole data for traning final models"""
    data["date"] = pd.to_datetime(data["date"], format="%Y-%m-%d") # date time formate
    final_model_dict = {}
    col_names = data.columns
    for y_indx in range(1, len(data.columns)):
        item_name = col_names[y_indx] # item key for model_dict
        with mlflow.start_run(run_name=f"{item_name}_run") as run:
            tsdf = data.iloc[:, [0, y_indx]]
            tsdf.columns = ["ds", "y"] # NP formate colums
            # model traning
            npf = Prophet() 
            npf.fit(tsdf)
            final_model_dict[item_name] = npf
              
            """mlflow tracking metrics and model item wise i.e. 50"""
            mlflow.prophet.log_model(npf, f"{item_name}_model") # we need model train on whole data not just on train data
            """eval metrics from traning and testing model"""
            dict_eval = item_metrics[item_name]
            mlflow.log_metric("MAPE_train",dict_eval["MAPE_train"])
            mlflow.log_metric("MAPE_test",dict_eval["MAPE_test"])
    return final_model_dict

# Phase 4 : mlflow run analysis and select best 50

In [8]:
def best_models():
    list_run = mlflow.search_runs() # tags.mlflow.runName base comare model and select final 
    runName_list = set(list_run["tags.mlflow.runName"])
    best_model_runs = {}
    for run_name in runName_list:
        df_item = list_run[list_run["tags.mlflow.runName"]==run_name]
        df_item = df_item.reset_index()
        df_item = df_item.iloc[: , 1:]
        df_item = df_item.sort_values("metrics.MAPE_test")
        best_model_runs[run_name] = df_item.loc[0]["run_id"]
    return best_model_runs

# Phase 5 : Forecasting next month with selected model runs

In [9]:
def forecast_with_run_model(run_model_dict, latest_proc_data, next_month_count = 1):
    """create dataframe for next month forecasting with current data used for traning model"""
#     date_last = list(latest_proc_data["date"])[-1]
#     strat = date_last + timedelta(1)
#     end = date_last + relativedelta(months=next_month_count)
#     data_next_month = pd.date_range(start= strat, end= end , freq="D")
#     data_next_month_df = pd.DataFrame({'ds':data_next_month})
    
    year = list(latest_proc_data["date"])[-1].year
    
    month = list(latest_proc_data["date"])[-1].month + next_month_count
    if month > 12 :
        month = 1
        year = year + 1
        
    last_day = calendar.monthrange(year , month)[1]
    start = datetime(year, month, 1)
    end = start + timedelta(last_day-1)
    data_next_month = pd.date_range(start= start, end= end , freq="D")
    data_next_month_df = pd.DataFrame({'ds':data_next_month})
    
    """fetch each best model from dict and forecast"""
    dict_fourcast = {}
    for name,run_id in run_model_dict.items():
        model_folder_name = f"{name[:-3]}model"
        pyfunc_uri = f"runs:/{run_id}/{model_folder_name}" # need to provide dynamic sales_7_model i.e item name folder here
        pyfunc_model = mlflow.pyfunc.load_model(pyfunc_uri) # we are using profet flavor    
        pred_df = pyfunc_model.predict(data_next_month_df)
        
        """final single datafarem with each item sales for next month"""
        name_col = name[:-4]
        dict_fourcast[name_col] = pred_df["yhat"].values
        df_fourcast = pd.DataFrame(dict_fourcast)
        final_fourcast = pd.concat([data_next_month_df["ds"], df_fourcast], axis=1)
        final_fourcast.rename(columns = {"ds" : "date"}, inplace = True)
    """make dataframe in standered formate"""
    col_name_order = ["date"]
    col_name_order.extend([f"sales_{i}" for i in range(1,51)])
    final_fourcast = final_fourcast.reindex(col_name_order, axis=1)
    final_fourcast = final_fourcast.round()
    final_fourcast.columns = ["date"] + [f"sales_{i}_forecast" for i in range(1,51)]
    return final_fourcast

In [ ]:
year = list(latest_proc_data["date"])[-1].year

month = list(latest_proc_data["date"])[-1].month + next_month_count 
if month > 12 :
    month = 1
    year = year + 1

last_day = calendar.monthrange(year , month)[1]
start = datetime(year, month, 1)
end = start + timedelta(last_day-1)

# Phase 6 : Model training and forecasting 

In [10]:
def model_train_forecast(base_data, next_month_data):
    """current data"""
    proc_data = pre_pros(base_data)
    dict_models = model_build_NP(proc_data, 0.1)
    final_models = final_model_build_NP(proc_data)
    run_model_dict = best_models()
    forecasted_month = forecast_with_run_model(run_model_dict, proc_data, 1)
    
    """next month data"""
    proc_data_month = pre_pros(next_month_data)
    total_no_days = int(list(proc_data_month["date"])[-1][-2:])
    true_proc_month = proc_data_month[-total_no_days:] 
    true_proc_month = true_proc_month.reset_index().iloc[:,1:]
    
    month_perfom = pd.concat([forecasted_month, true_proc_month.drop("date", axis=1)], axis=1)
    
    col_name_convi = ["date"]
    for i in range(1,51):
        col_name_convi.extend([f"sales_{i}", f"sales_{i}_forecast"]) 
    month_perfom = month_perfom.reindex(columns=col_name_convi) # true and forcast data for month

    df = pd.DataFrame() # diff between true and forcasted dataframe
    fore_mon = forecasted_month.drop("date", axis=1)
    true_mon = true_proc_month.drop("date", axis=1)
    for i,j in zip(list(fore_mon.columns),list(true_mon.columns)):
        df[f"{i}_diff"] = forecasted_month[i] - true_proc_month[j]
    
    acuracy = 1- mean_absolute_percentage_error(fore_mon, true_mon)
    return {"acuracy":acuracy, "diff_df" : df, "true_forecat_df" : month_perfom}

# testing data

## updated 2013 to 2016 data model and its forecasting 

In [11]:
dict_jan = model_train_forecast(base_data, jan_month_data)

14:50:02 - cmdstanpy - INFO - Chain [1] start processing
14:50:02 - cmdstanpy - INFO - Chain [1] done processing
14:50:03 - cmdstanpy - INFO - Chain [1] start processing
14:50:03 - cmdstanpy - INFO - Chain [1] done processing
14:50:04 - cmdstanpy - INFO - Chain [1] start processing
14:50:04 - cmdstanpy - INFO - Chain [1] done processing
14:50:05 - cmdstanpy - INFO - Chain [1] start processing
14:50:05 - cmdstanpy - INFO - Chain [1] done processing
14:50:05 - cmdstanpy - INFO - Chain [1] start processing
14:50:06 - cmdstanpy - INFO - Chain [1] done processing
14:50:06 - cmdstanpy - INFO - Chain [1] start processing
14:50:06 - cmdstanpy - INFO - Chain [1] done processing
14:50:07 - cmdstanpy - INFO - Chain [1] start processing
14:50:07 - cmdstanpy - INFO - Chain [1] done processing
14:50:08 - cmdstanpy - INFO - Chain [1] start processing
14:50:08 - cmdstanpy - INFO - Chain [1] done processing
14:50:09 - cmdstanpy - INFO - Chain [1] start processing
14:50:09 - cmdstanpy - INFO - Chain [1]

14:50:50 - cmdstanpy - INFO - Chain [1] done processing
14:50:50 - cmdstanpy - INFO - Chain [1] start processing
14:50:50 - cmdstanpy - INFO - Chain [1] done processing
14:50:51 - cmdstanpy - INFO - Chain [1] start processing
14:50:51 - cmdstanpy - INFO - Chain [1] done processing
14:50:51 - cmdstanpy - INFO - Chain [1] start processing
14:50:51 - cmdstanpy - INFO - Chain [1] done processing
14:50:51 - cmdstanpy - INFO - Chain [1] start processing
14:50:51 - cmdstanpy - INFO - Chain [1] done processing
14:50:52 - cmdstanpy - INFO - Chain [1] start processing
14:50:52 - cmdstanpy - INFO - Chain [1] done processing
14:50:52 - cmdstanpy - INFO - Chain [1] start processing
14:50:52 - cmdstanpy - INFO - Chain [1] done processing
14:50:52 - cmdstanpy - INFO - Chain [1] start processing
14:50:53 - cmdstanpy - INFO - Chain [1] done processing
14:50:53 - cmdstanpy - INFO - Chain [1] start processing
14:50:53 - cmdstanpy - INFO - Chain [1] done processing
14:50:53 - cmdstanpy - INFO - Chain [1] 

In [12]:
dict_jan["acuracy"]

0.834885401983703

In [13]:
dict_jan["diff_df"]

,sales_1_forecast_diff,sales_2_forecast_diff,sales_3_forecast_diff,sales_4_forecast_diff,sales_5_forecast_diff,sales_6_forecast_diff,sales_7_forecast_diff,sales_8_forecast_diff,sales_9_forecast_diff,sales_10_forecast_diff,...,sales_41_forecast_diff,sales_42_forecast_diff,sales_43_forecast_diff,sales_44_forecast_diff,sales_45_forecast_diff,sales_46_forecast_diff,sales_47_forecast_diff,sales_48_forecast_diff,sales_49_forecast_diff,sales_50_forecast_diff
0,2.0,17.0,-2.0,0.0,-8.0,5.0,11.0,9.0,5.0,16.0,...,-9.0,4.0,6.0,-6.0,19.0,18.0,3.0,3.0,3.0,12.0
1,-2.0,10.0,-2.0,-2.0,-3.0,7.0,-4.0,11.0,4.0,-8.0,...,-5.0,4.0,1.0,0.0,8.0,3.0,5.0,1.0,-6.0,2.0
2,6.0,10.0,5.0,4.0,0.0,13.0,8.0,11.0,9.0,-2.0,...,5.0,1.0,8.0,-6.0,10.0,2.0,-10.0,-7.0,-7.0,9.0
3,0.0,2.0,2.0,2.0,-2.0,-6.0,-3.0,6.0,-5.0,6.0,...,-2.0,4.0,2.0,6.0,-5.0,4.0,-2.0,2.0,3.0,3.0
4,3.0,7.0,3.0,8.0,0.0,6.0,-1.0,19.0,4.0,10.0,...,2.0,-7.0,-6.0,-3.0,0.0,8.0,2.0,0.0,5.0,-4.0
5,-5.0,7.0,7.0,8.0,-3.0,17.0,11.0,5.0,10.0,8.0,...,0.0,2.0,2.0,3.0,15.0,3.0,6.0,3.0,-10.0,-2.0
6,6.0,7.0,6.0,-1.0,1.0,11.0,8.0,-15.0,7.0,3.0,...,3.0,4.0,2.0,-6.0,3.0,-1.0,4.0,7.0,2.0,12.0
7,1.0,-1.0,3.0,5.0,5.0,6.0,10.0,17.0,16.0,6.0,...,1.0,0.0,8.0,8.0,7.0,6.0,2.0,2.0,-6.0,13.0
8,-5.0,5.0,-1.0,8.0,-6.0,-4.0,7.0,4.0,4.0,8.0,...,5.0,7.0,9.0,1.0,10.0,0.0,-1.0,3.0,5.0,4.0
9,5.0,7.0,9.0,-2.0,2.0,4.0,7.0,10.0,9.0,-2.0,...,1.0,4.0,-2.0,6.0,14.0,3.0,2.0,3.0,-1.0,-3.0


In [14]:
dict_jan["true_forecat_df"]

,date,sales_1,sales_1_forecast,sales_2,sales_2_forecast,sales_3,sales_3_forecast,sales_4,sales_4_forecast,sales_5,...,sales_46,sales_46_forecast,sales_47,sales_47_forecast,sales_48,sales_48_forecast,sales_49,sales_49_forecast,sales_50,sales_50_forecast
0,2017-01-01,19,21.0,40,57.0,36,34.0,21,21.0,25,...,38,56.0,17,20.0,45,48.0,24,27.0,49,61.0
1,2017-01-02,15,13.0,26,36.0,23,21.0,15,13.0,14,...,32,35.0,8,13.0,29,30.0,23,17.0,36,38.0
2,2017-01-03,10,16.0,33,43.0,20,25.0,12,16.0,13,...,39,41.0,26,16.0,43,36.0,27,20.0,37,46.0
3,2017-01-04,16,16.0,41,43.0,23,25.0,14,16.0,15,...,38,42.0,18,16.0,34,36.0,17,20.0,44,47.0
4,2017-01-05,14,17.0,39,46.0,25,28.0,10,18.0,15,...,38,46.0,15,17.0,39,39.0,18,23.0,54,50.0
5,2017-01-06,24,19.0,43,50.0,23,30.0,10,18.0,18,...,46,49.0,12,18.0,39,42.0,33,23.0,55,53.0
6,2017-01-07,14,20.0,46,53.0,26,32.0,21,20.0,15,...,52,51.0,15,19.0,38,45.0,23,25.0,46,58.0
7,2017-01-08,20,21.0,58,57.0,32,35.0,16,21.0,12,...,50,56.0,18,20.0,46,48.0,33,27.0,48,61.0
8,2017-01-09,18,13.0,31,36.0,23,22.0,5,13.0,17,...,35,35.0,14,13.0,27,30.0,12,17.0,35,39.0
9,2017-01-10,11,16.0,36,43.0,17,26.0,18,16.0,11,...,39,42.0,13,15.0,33,36.0,21,20.0,49,46.0


## updated jan 2017 data model and its forecasting 

In [15]:
dict_feb = model_train_forecast(jan_month_data, feb_month_data)

14:51:31 - cmdstanpy - INFO - Chain [1] start processing
14:51:31 - cmdstanpy - INFO - Chain [1] done processing
14:51:32 - cmdstanpy - INFO - Chain [1] start processing
14:51:32 - cmdstanpy - INFO - Chain [1] done processing
14:51:32 - cmdstanpy - INFO - Chain [1] start processing
14:51:33 - cmdstanpy - INFO - Chain [1] done processing
14:51:33 - cmdstanpy - INFO - Chain [1] start processing
14:51:33 - cmdstanpy - INFO - Chain [1] done processing
14:51:34 - cmdstanpy - INFO - Chain [1] start processing
14:51:34 - cmdstanpy - INFO - Chain [1] done processing
14:51:35 - cmdstanpy - INFO - Chain [1] start processing
14:51:35 - cmdstanpy - INFO - Chain [1] done processing
14:51:36 - cmdstanpy - INFO - Chain [1] start processing
14:51:36 - cmdstanpy - INFO - Chain [1] done processing
14:51:37 - cmdstanpy - INFO - Chain [1] start processing
14:51:37 - cmdstanpy - INFO - Chain [1] done processing
14:51:38 - cmdstanpy - INFO - Chain [1] start processing
14:51:38 - cmdstanpy - INFO - Chain [1]

14:52:22 - cmdstanpy - INFO - Chain [1] done processing
14:52:22 - cmdstanpy - INFO - Chain [1] start processing
14:52:23 - cmdstanpy - INFO - Chain [1] done processing
14:52:23 - cmdstanpy - INFO - Chain [1] start processing
14:52:23 - cmdstanpy - INFO - Chain [1] done processing
14:52:23 - cmdstanpy - INFO - Chain [1] start processing
14:52:23 - cmdstanpy - INFO - Chain [1] done processing
14:52:24 - cmdstanpy - INFO - Chain [1] start processing
14:52:24 - cmdstanpy - INFO - Chain [1] done processing
14:52:24 - cmdstanpy - INFO - Chain [1] start processing
14:52:24 - cmdstanpy - INFO - Chain [1] done processing
14:52:24 - cmdstanpy - INFO - Chain [1] start processing
14:52:24 - cmdstanpy - INFO - Chain [1] done processing
14:52:24 - cmdstanpy - INFO - Chain [1] start processing
14:52:25 - cmdstanpy - INFO - Chain [1] done processing
14:52:25 - cmdstanpy - INFO - Chain [1] start processing
14:52:25 - cmdstanpy - INFO - Chain [1] done processing
14:52:25 - cmdstanpy - INFO - Chain [1] 

In [16]:
dict_feb["acuracy"]

0.8579127660028217

In [17]:
dict_feb["diff_df"]

,sales_1_forecast_diff,sales_2_forecast_diff,sales_3_forecast_diff,sales_4_forecast_diff,sales_5_forecast_diff,sales_6_forecast_diff,sales_7_forecast_diff,sales_8_forecast_diff,sales_9_forecast_diff,sales_10_forecast_diff,...,sales_41_forecast_diff,sales_42_forecast_diff,sales_43_forecast_diff,sales_44_forecast_diff,sales_45_forecast_diff,sales_46_forecast_diff,sales_47_forecast_diff,sales_48_forecast_diff,sales_49_forecast_diff,sales_50_forecast_diff
0,1.0,7.0,1.0,1.0,-6.0,-10.0,-12.0,-2.0,-1.0,-1.0,...,-2.0,1.0,-5.0,-6.0,1.0,8.0,2.0,-7.0,5.0,-15.0
1,0.0,-3.0,8.0,1.0,5.0,-1.0,6.0,3.0,3.0,0.0,...,9.0,-5.0,7.0,0.0,-2.0,7.0,0.0,4.0,5.0,-1.0
2,-5.0,3.0,-1.0,-1.0,3.0,3.0,-1.0,-5.0,4.0,6.0,...,0.0,2.0,-1.0,-4.0,-8.0,-5.0,2.0,7.0,1.0,6.0
3,-5.0,9.0,13.0,2.0,2.0,9.0,10.0,16.0,-3.0,12.0,...,-2.0,0.0,5.0,3.0,-11.0,-4.0,1.0,-5.0,-2.0,6.0
4,-4.0,17.0,-3.0,2.0,4.0,7.0,-6.0,8.0,-3.0,4.0,...,-1.0,1.0,5.0,-1.0,1.0,1.0,3.0,1.0,-3.0,-1.0
5,-3.0,-3.0,0.0,1.0,-6.0,-2.0,-5.0,-2.0,-9.0,-2.0,...,0.0,-2.0,-4.0,0.0,-3.0,2.0,0.0,-4.0,0.0,2.0
6,4.0,9.0,-1.0,4.0,-5.0,3.0,0.0,8.0,12.0,-8.0,...,3.0,2.0,4.0,1.0,9.0,0.0,-1.0,8.0,3.0,-7.0
7,-4.0,6.0,7.0,4.0,0.0,11.0,4.0,5.0,-3.0,3.0,...,8.0,0.0,3.0,4.0,8.0,4.0,3.0,1.0,-4.0,-4.0
8,1.0,7.0,-1.0,11.0,1.0,-5.0,-1.0,-1.0,13.0,3.0,...,3.0,2.0,0.0,0.0,-8.0,-1.0,4.0,-7.0,9.0,-2.0
9,3.0,11.0,-6.0,-1.0,10.0,6.0,5.0,9.0,-19.0,9.0,...,3.0,7.0,3.0,-3.0,9.0,7.0,0.0,2.0,3.0,13.0


In [18]:
dict_feb["true_forecat_df"]

,date,sales_1,sales_1_forecast,sales_2,sales_2_forecast,sales_3,sales_3_forecast,sales_4,sales_4_forecast,sales_5,...,sales_46,sales_46_forecast,sales_47,sales_47_forecast,sales_48,sales_48_forecast,sales_49,sales_49_forecast,sales_50,sales_50_forecast
0,2017-02-01,15,16.0,34,41.0,25,26.0,15,16.0,19,...,33,41.0,14,16.0,43,36.0,15,20.0,62,47.0
1,2017-02-02,17,17.0,47,44.0,20,28.0,17,18.0,10,...,38,45.0,18,18.0,35,39.0,18,23.0,51,50.0
2,2017-02-03,24,19.0,45,48.0,31,30.0,19,18.0,12,...,53,48.0,16,18.0,36,43.0,23,24.0,48,54.0
3,2017-02-04,25,20.0,42,51.0,19,32.0,18,20.0,14,...,55,51.0,19,20.0,50,45.0,28,26.0,53,59.0
4,2017-02-05,25,21.0,38,55.0,38,35.0,20,22.0,13,...,54,55.0,18,21.0,48,49.0,31,28.0,63,62.0
5,2017-02-06,16,13.0,38,35.0,23,23.0,13,14.0,17,...,33,35.0,14,14.0,35,31.0,18,18.0,38,40.0
6,2017-02-07,12,16.0,33,42.0,27,26.0,13,17.0,19,...,42,42.0,18,17.0,29,37.0,18,21.0,54,47.0
7,2017-02-08,20,16.0,37,43.0,20,27.0,13,17.0,14,...,38,42.0,14,17.0,36,37.0,25,21.0,53,49.0
8,2017-02-09,16,17.0,39,46.0,30,29.0,8,19.0,14,...,47,46.0,14,18.0,47,40.0,15,24.0,54,52.0
9,2017-02-10,16,19.0,38,49.0,37,31.0,20,19.0,6,...,43,50.0,19,19.0,42,44.0,21,24.0,43,56.0


## updated feb 2017 data model and its forecasting 

In [19]:
dict_mar = model_train_forecast(feb_month_data, mar_month_data)

14:52:49 - cmdstanpy - INFO - Chain [1] start processing
14:52:49 - cmdstanpy - INFO - Chain [1] done processing
14:52:50 - cmdstanpy - INFO - Chain [1] start processing
14:52:50 - cmdstanpy - INFO - Chain [1] done processing
14:52:51 - cmdstanpy - INFO - Chain [1] start processing
14:52:51 - cmdstanpy - INFO - Chain [1] done processing
14:52:52 - cmdstanpy - INFO - Chain [1] start processing
14:52:52 - cmdstanpy - INFO - Chain [1] done processing
14:52:52 - cmdstanpy - INFO - Chain [1] start processing
14:52:53 - cmdstanpy - INFO - Chain [1] done processing
14:52:53 - cmdstanpy - INFO - Chain [1] start processing
14:52:53 - cmdstanpy - INFO - Chain [1] done processing
14:52:54 - cmdstanpy - INFO - Chain [1] start processing
14:52:54 - cmdstanpy - INFO - Chain [1] done processing
14:52:55 - cmdstanpy - INFO - Chain [1] start processing
14:52:55 - cmdstanpy - INFO - Chain [1] done processing
14:52:56 - cmdstanpy - INFO - Chain [1] start processing
14:52:56 - cmdstanpy - INFO - Chain [1]

14:53:43 - cmdstanpy - INFO - Chain [1] done processing
14:53:43 - cmdstanpy - INFO - Chain [1] start processing
14:53:43 - cmdstanpy - INFO - Chain [1] done processing
14:53:43 - cmdstanpy - INFO - Chain [1] start processing
14:53:43 - cmdstanpy - INFO - Chain [1] done processing
14:53:44 - cmdstanpy - INFO - Chain [1] start processing
14:53:44 - cmdstanpy - INFO - Chain [1] done processing
14:53:44 - cmdstanpy - INFO - Chain [1] start processing
14:53:44 - cmdstanpy - INFO - Chain [1] done processing
14:53:44 - cmdstanpy - INFO - Chain [1] start processing
14:53:45 - cmdstanpy - INFO - Chain [1] done processing
14:53:45 - cmdstanpy - INFO - Chain [1] start processing
14:53:45 - cmdstanpy - INFO - Chain [1] done processing
14:53:45 - cmdstanpy - INFO - Chain [1] start processing
14:53:45 - cmdstanpy - INFO - Chain [1] done processing
14:53:45 - cmdstanpy - INFO - Chain [1] start processing
14:53:46 - cmdstanpy - INFO - Chain [1] done processing
14:53:46 - cmdstanpy - INFO - Chain [1] 

In [20]:
dict_mar["acuracy"]

0.8660157733334384

In [23]:
dict_mar["diff_df"]

,sales_1_forecast_diff,sales_2_forecast_diff,sales_3_forecast_diff,sales_4_forecast_diff,sales_5_forecast_diff,sales_6_forecast_diff,sales_7_forecast_diff,sales_8_forecast_diff,sales_9_forecast_diff,sales_10_forecast_diff,...,sales_41_forecast_diff,sales_42_forecast_diff,sales_43_forecast_diff,sales_44_forecast_diff,sales_45_forecast_diff,sales_46_forecast_diff,sales_47_forecast_diff,sales_48_forecast_diff,sales_49_forecast_diff,sales_50_forecast_diff
0,1.0,-2.0,-12.0,-2.0,-1.0,-19.0,-10.0,-9.0,-3.0,-5.0,...,-2.0,-11.0,-11.0,-5.0,-3.0,-9.0,4.0,6.0,-3.0,0.0
1,-5.0,-5.0,-4.0,-4.0,1.0,8.0,-2.0,-7.0,-3.0,4.0,...,-6.0,-10.0,-1.0,-2.0,-10.0,-3.0,-5.0,-3.0,-16.0,1.0
2,-2.0,5.0,1.0,-5.0,4.0,-7.0,-7.0,-13.0,-1.0,12.0,...,-2.0,-9.0,11.0,-11.0,-7.0,-12.0,-2.0,0.0,4.0,-20.0
3,9.0,-14.0,2.0,7.0,2.0,-8.0,1.0,7.0,1.0,7.0,...,-8.0,-11.0,-3.0,-10.0,-5.0,-6.0,6.0,10.0,-7.0,-6.0
4,-2.0,3.0,-4.0,7.0,-6.0,-11.0,2.0,-11.0,-1.0,-6.0,...,0.0,-1.0,-3.0,-4.0,6.0,-5.0,-4.0,5.0,-1.0,-9.0
5,4.0,4.0,6.0,3.0,9.0,-2.0,4.0,2.0,4.0,-18.0,...,5.0,-7.0,7.0,-4.0,-1.0,-6.0,6.0,5.0,3.0,-10.0
6,4.0,-10.0,-1.0,0.0,7.0,-4.0,8.0,-4.0,11.0,-2.0,...,2.0,5.0,2.0,-7.0,-8.0,1.0,0.0,-9.0,3.0,0.0
7,2.0,-3.0,-6.0,0.0,0.0,-1.0,-15.0,-3.0,1.0,6.0,...,-5.0,-6.0,4.0,7.0,-4.0,-7.0,1.0,2.0,-1.0,3.0
8,4.0,8.0,4.0,-5.0,-1.0,2.0,-1.0,-26.0,0.0,11.0,...,1.0,-2.0,1.0,2.0,-18.0,-11.0,8.0,-9.0,-4.0,-13.0
9,4.0,1.0,-2.0,-4.0,-10.0,-14.0,-16.0,-5.0,8.0,0.0,...,4.0,1.0,6.0,7.0,7.0,5.0,9.0,-5.0,-10.0,0.0


In [24]:
dict_mar["true_forecat_df"]

,date,sales_1,sales_1_forecast,sales_2,sales_2_forecast,sales_3,sales_3_forecast,sales_4,sales_4_forecast,sales_5,...,sales_46,sales_46_forecast,sales_47,sales_47_forecast,sales_48,sales_48_forecast,sales_49,sales_49_forecast,sales_50,sales_50_forecast
0,2017-03-01,16,17.0,48,46.0,40,28.0,19,17.0,17,...,55,46.0,14,18.0,36,42.0,25,22.0,52,52.0
1,2017-03-02,23,18.0,54,49.0,34,30.0,23,19.0,16,...,52,49.0,24,19.0,48,45.0,41,25.0,54,55.0
2,2017-03-03,22,20.0,47,52.0,32,33.0,25,20.0,14,...,65,53.0,22,20.0,49,49.0,22,26.0,78,58.0
3,2017-03-04,13,22.0,70,56.0,32,34.0,15,22.0,17,...,62,56.0,16,22.0,41,51.0,35,28.0,69,63.0
4,2017-03-05,25,23.0,57,60.0,41,37.0,16,23.0,26,...,65,60.0,27,23.0,50,55.0,32,31.0,75,66.0
5,2017-03-06,11,15.0,36,40.0,19,25.0,12,15.0,5,...,46,40.0,10,16.0,32,37.0,17,20.0,55,45.0
6,2017-03-07,14,18.0,57,47.0,30,29.0,18,18.0,9,...,45,46.0,19,19.0,53,44.0,21,24.0,52,52.0
7,2017-03-08,17,19.0,51,48.0,35,29.0,18,18.0,16,...,54,47.0,19,20.0,42,44.0,25,24.0,51,54.0
8,2017-03-09,16,20.0,43,51.0,27,31.0,25,20.0,19,...,62,51.0,13,21.0,56,47.0,30,26.0,70,57.0
9,2017-03-10,18,22.0,53,54.0,36,34.0,24,20.0,28,...,50,55.0,13,22.0,56,51.0,37,27.0,61,61.0


## updated march 2017 data model and its forecasting 

In [25]:
dict_apr = model_train_forecast(mar_month_data, apr_month_data)

14:55:20 - cmdstanpy - INFO - Chain [1] start processing
14:55:20 - cmdstanpy - INFO - Chain [1] done processing
14:55:21 - cmdstanpy - INFO - Chain [1] start processing
14:55:21 - cmdstanpy - INFO - Chain [1] done processing
14:55:22 - cmdstanpy - INFO - Chain [1] start processing
14:55:22 - cmdstanpy - INFO - Chain [1] done processing
14:55:23 - cmdstanpy - INFO - Chain [1] start processing
14:55:23 - cmdstanpy - INFO - Chain [1] done processing
14:55:23 - cmdstanpy - INFO - Chain [1] start processing
14:55:24 - cmdstanpy - INFO - Chain [1] done processing
14:55:24 - cmdstanpy - INFO - Chain [1] start processing
14:55:24 - cmdstanpy - INFO - Chain [1] done processing
14:55:25 - cmdstanpy - INFO - Chain [1] start processing
14:55:25 - cmdstanpy - INFO - Chain [1] done processing
14:55:26 - cmdstanpy - INFO - Chain [1] start processing
14:55:26 - cmdstanpy - INFO - Chain [1] done processing
14:55:27 - cmdstanpy - INFO - Chain [1] start processing
14:55:27 - cmdstanpy - INFO - Chain [1]

14:56:13 - cmdstanpy - INFO - Chain [1] done processing
14:56:13 - cmdstanpy - INFO - Chain [1] start processing
14:56:13 - cmdstanpy - INFO - Chain [1] done processing
14:56:13 - cmdstanpy - INFO - Chain [1] start processing
14:56:13 - cmdstanpy - INFO - Chain [1] done processing
14:56:14 - cmdstanpy - INFO - Chain [1] start processing
14:56:14 - cmdstanpy - INFO - Chain [1] done processing
14:56:14 - cmdstanpy - INFO - Chain [1] start processing
14:56:14 - cmdstanpy - INFO - Chain [1] done processing
14:56:14 - cmdstanpy - INFO - Chain [1] start processing
14:56:14 - cmdstanpy - INFO - Chain [1] done processing
14:56:15 - cmdstanpy - INFO - Chain [1] start processing
14:56:15 - cmdstanpy - INFO - Chain [1] done processing
14:56:15 - cmdstanpy - INFO - Chain [1] start processing
14:56:15 - cmdstanpy - INFO - Chain [1] done processing
14:56:15 - cmdstanpy - INFO - Chain [1] start processing
14:56:16 - cmdstanpy - INFO - Chain [1] done processing
14:56:16 - cmdstanpy - INFO - Chain [1] 

In [26]:
dict_apr["acuracy"]

0.8735789301866302

In [27]:
dict_apr["diff_df"]

,sales_1_forecast_diff,sales_2_forecast_diff,sales_3_forecast_diff,sales_4_forecast_diff,sales_5_forecast_diff,sales_6_forecast_diff,sales_7_forecast_diff,sales_8_forecast_diff,sales_9_forecast_diff,sales_10_forecast_diff,...,sales_41_forecast_diff,sales_42_forecast_diff,sales_43_forecast_diff,sales_44_forecast_diff,sales_45_forecast_diff,sales_46_forecast_diff,sales_47_forecast_diff,sales_48_forecast_diff,sales_49_forecast_diff,sales_50_forecast_diff
0,-2.0,-1.0,-1.0,-12.0,-5.0,-8.0,-22.0,-22.0,-17.0,-4.0,...,-3.0,2.0,-17.0,-5.0,-7.0,-5.0,2.0,-13.0,-8.0,0.0
1,4.0,-3.0,-1.0,-2.0,-10.0,-19.0,-5.0,12.0,-6.0,-11.0,...,-1.0,-3.0,-10.0,-3.0,-14.0,-24.0,-8.0,-5.0,-3.0,-6.0
2,0.0,1.0,-3.0,-1.0,-2.0,-4.0,-5.0,-13.0,-8.0,-10.0,...,-2.0,3.0,1.0,1.0,4.0,14.0,-7.0,3.0,-5.0,-4.0
3,-2.0,-2.0,1.0,3.0,-3.0,-12.0,0.0,9.0,-9.0,-16.0,...,-4.0,-15.0,6.0,-11.0,1.0,-5.0,-4.0,-14.0,5.0,3.0
4,-1.0,-24.0,-8.0,5.0,1.0,-3.0,0.0,4.0,-8.0,-11.0,...,2.0,-4.0,-1.0,0.0,-7.0,-4.0,-1.0,-16.0,-8.0,8.0
5,-4.0,-7.0,-4.0,-5.0,1.0,9.0,-2.0,-9.0,2.0,1.0,...,3.0,-12.0,-8.0,-1.0,-15.0,-9.0,-3.0,5.0,-15.0,-5.0
6,-2.0,-4.0,-7.0,6.0,5.0,4.0,7.0,-16.0,0.0,-11.0,...,0.0,-1.0,-2.0,-4.0,-13.0,-6.0,-2.0,4.0,2.0,-2.0
7,1.0,-15.0,8.0,-2.0,-4.0,-19.0,-19.0,-10.0,-5.0,-18.0,...,1.0,-3.0,13.0,1.0,-29.0,-2.0,-3.0,-17.0,8.0,8.0
8,6.0,-20.0,1.0,4.0,-2.0,-15.0,1.0,-5.0,-3.0,0.0,...,-2.0,-19.0,-10.0,-2.0,-17.0,1.0,-3.0,-13.0,-8.0,4.0
9,-1.0,-6.0,-1.0,0.0,7.0,-7.0,1.0,6.0,-1.0,-5.0,...,4.0,5.0,-8.0,0.0,9.0,-6.0,-8.0,9.0,-5.0,-3.0


In [29]:
dict_apr["true_forecat_df"]

,date,sales_1,sales_1_forecast,sales_2,sales_2_forecast,sales_3,sales_3_forecast,sales_4,sales_4_forecast,sales_5,...,sales_46,sales_46_forecast,sales_47,sales_47_forecast,sales_48,sales_48_forecast,sales_49,sales_49_forecast,sales_50,sales_50_forecast
0,2017-04-01,26,24.0,61,60.0,40,39.0,37,25.0,26,...,68,63.0,21,23.0,68,55.0,40,32.0,72,72.0
1,2017-04-02,21,25.0,66,63.0,43,42.0,28,26.0,32,...,91,67.0,32,24.0,65,60.0,38,35.0,81,75.0
2,2017-04-03,17,17.0,43,44.0,33,30.0,19,18.0,18,...,33,47.0,24,17.0,38,41.0,30,25.0,58,54.0
3,2017-04-04,22,20.0,53,51.0,32,33.0,18,21.0,21,...,58,53.0,24,20.0,62,48.0,23,28.0,59,62.0
4,2017-04-05,21,20.0,76,52.0,42,34.0,16,21.0,17,...,59,55.0,21,20.0,64,48.0,36,28.0,55,63.0
5,2017-04-06,25,21.0,62,55.0,40,36.0,28,23.0,19,...,68,59.0,24,21.0,46,51.0,46,31.0,71,66.0
6,2017-04-07,25,23.0,62,58.0,45,38.0,18,24.0,15,...,68,62.0,24,22.0,51,55.0,29,31.0,72,70.0
7,2017-04-08,23,24.0,77,62.0,32,40.0,28,26.0,25,...,67,65.0,26,23.0,74,57.0,25,33.0,67,75.0
8,2017-04-09,20,26.0,86,66.0,42,43.0,23,27.0,24,...,69,70.0,28,25.0,74,61.0,44,36.0,74,78.0
9,2017-04-10,19,18.0,52,46.0,31,30.0,19,19.0,9,...,56,50.0,25,17.0,34,43.0,31,26.0,59,56.0


## updated april 2017 data model and its forecasting 

In [30]:
dict_may = model_train_forecast(apr_month_data, may_month_data)

14:57:07 - cmdstanpy - INFO - Chain [1] start processing
14:57:07 - cmdstanpy - INFO - Chain [1] done processing
14:57:08 - cmdstanpy - INFO - Chain [1] start processing
14:57:08 - cmdstanpy - INFO - Chain [1] done processing
14:57:09 - cmdstanpy - INFO - Chain [1] start processing
14:57:09 - cmdstanpy - INFO - Chain [1] done processing
14:57:10 - cmdstanpy - INFO - Chain [1] start processing
14:57:10 - cmdstanpy - INFO - Chain [1] done processing
14:57:11 - cmdstanpy - INFO - Chain [1] start processing
14:57:11 - cmdstanpy - INFO - Chain [1] done processing
14:57:12 - cmdstanpy - INFO - Chain [1] start processing
14:57:12 - cmdstanpy - INFO - Chain [1] done processing
14:57:13 - cmdstanpy - INFO - Chain [1] start processing
14:57:13 - cmdstanpy - INFO - Chain [1] done processing
14:57:13 - cmdstanpy - INFO - Chain [1] start processing
14:57:13 - cmdstanpy - INFO - Chain [1] done processing
14:57:14 - cmdstanpy - INFO - Chain [1] start processing
14:57:14 - cmdstanpy - INFO - Chain [1]

14:58:02 - cmdstanpy - INFO - Chain [1] done processing
14:58:02 - cmdstanpy - INFO - Chain [1] start processing
14:58:02 - cmdstanpy - INFO - Chain [1] done processing
14:58:02 - cmdstanpy - INFO - Chain [1] start processing
14:58:02 - cmdstanpy - INFO - Chain [1] done processing
14:58:03 - cmdstanpy - INFO - Chain [1] start processing
14:58:03 - cmdstanpy - INFO - Chain [1] done processing
14:58:03 - cmdstanpy - INFO - Chain [1] start processing
14:58:03 - cmdstanpy - INFO - Chain [1] done processing
14:58:03 - cmdstanpy - INFO - Chain [1] start processing
14:58:04 - cmdstanpy - INFO - Chain [1] done processing
14:58:04 - cmdstanpy - INFO - Chain [1] start processing
14:58:04 - cmdstanpy - INFO - Chain [1] done processing
14:58:04 - cmdstanpy - INFO - Chain [1] start processing
14:58:04 - cmdstanpy - INFO - Chain [1] done processing
14:58:05 - cmdstanpy - INFO - Chain [1] start processing
14:58:05 - cmdstanpy - INFO - Chain [1] done processing
14:58:05 - cmdstanpy - INFO - Chain [1] 

In [31]:
dict_may["acuracy"]

0.8784401297428432

In [32]:
dict_may["diff_df"]

,sales_1_forecast_diff,sales_2_forecast_diff,sales_3_forecast_diff,sales_4_forecast_diff,sales_5_forecast_diff,sales_6_forecast_diff,sales_7_forecast_diff,sales_8_forecast_diff,sales_9_forecast_diff,sales_10_forecast_diff,...,sales_41_forecast_diff,sales_42_forecast_diff,sales_43_forecast_diff,sales_44_forecast_diff,sales_45_forecast_diff,sales_46_forecast_diff,sales_47_forecast_diff,sales_48_forecast_diff,sales_49_forecast_diff,sales_50_forecast_diff
0,4.0,-8.0,10.0,7.0,5.0,11.0,-2.0,7.0,-9.0,0.0,...,2.0,5.0,6.0,-4.0,6.0,-4.0,2.0,0.0,-5.0,15.0
1,0.0,-7.0,-7.0,-3.0,2.0,0.0,-13.0,-10.0,-11.0,-12.0,...,-3.0,-1.0,-11.0,2.0,-5.0,-9.0,3.0,-3.0,2.0,-6.0
2,-7.0,8.0,-10.0,-1.0,1.0,-14.0,-20.0,-19.0,-1.0,1.0,...,0.0,3.0,3.0,-12.0,14.0,-5.0,-11.0,-15.0,-8.0,-8.0
3,3.0,-1.0,-2.0,7.0,-1.0,1.0,4.0,7.0,4.0,-4.0,...,-2.0,8.0,0.0,10.0,9.0,-13.0,0.0,-8.0,-1.0,-15.0
4,-4.0,-14.0,-8.0,2.0,0.0,8.0,2.0,-14.0,8.0,9.0,...,1.0,-2.0,-11.0,0.0,4.0,1.0,-14.0,-1.0,-11.0,-4.0
5,-9.0,-7.0,1.0,0.0,6.0,6.0,10.0,-19.0,-5.0,5.0,...,-7.0,-1.0,-21.0,-3.0,6.0,-13.0,-3.0,-13.0,-9.0,-24.0
6,-4.0,6.0,-18.0,-5.0,-1.0,6.0,-11.0,8.0,1.0,-16.0,...,-9.0,7.0,4.0,2.0,0.0,-17.0,-1.0,-6.0,2.0,0.0
7,-5.0,-12.0,-1.0,10.0,-4.0,13.0,-6.0,-26.0,-5.0,-14.0,...,4.0,4.0,-1.0,-2.0,10.0,-2.0,-6.0,-6.0,6.0,-4.0
8,-11.0,-14.0,8.0,5.0,2.0,0.0,-1.0,-9.0,2.0,-2.0,...,4.0,5.0,-8.0,-7.0,-8.0,-1.0,5.0,-8.0,-9.0,-13.0
9,5.0,5.0,-5.0,9.0,2.0,5.0,4.0,-4.0,-2.0,1.0,...,5.0,3.0,-4.0,2.0,-19.0,3.0,-2.0,7.0,1.0,5.0


In [33]:
dict_may["true_forecat_df"]

,date,sales_1,sales_1_forecast,sales_2,sales_2_forecast,sales_3,sales_3_forecast,sales_4,sales_4_forecast,sales_5,...,sales_46,sales_46_forecast,sales_47,sales_47_forecast,sales_48,sales_48_forecast,sales_49,sales_49_forecast,sales_50,sales_50_forecast
0,2017-05-01,15,19.0,60,52.0,25,35.0,13,20.0,12,...,56,52.0,19,21.0,47,47.0,31,26.0,44,59.0
1,2017-05-02,21,21.0,66,59.0,45,38.0,26,23.0,17,...,67,58.0,20,23.0,57,54.0,27,29.0,72,66.0
2,2017-05-03,29,22.0,51,59.0,49,39.0,24,23.0,19,...,65,60.0,34,23.0,68,53.0,37,29.0,75,67.0
3,2017-05-04,19,22.0,63,62.0,43,41.0,18,25.0,22,...,76,63.0,25,25.0,64,56.0,33,32.0,85,70.0
4,2017-05-05,28,24.0,79,65.0,51,43.0,23,25.0,21,...,65,66.0,39,25.0,61,60.0,44,33.0,77,73.0
5,2017-05-06,34,25.0,76,69.0,44,45.0,27,27.0,16,...,82,69.0,30,27.0,75,62.0,44,35.0,102,78.0
6,2017-05-07,31,27.0,67,73.0,66,48.0,33,28.0,25,...,91,74.0,29,28.0,72,66.0,36,38.0,81,81.0
7,2017-05-08,24,19.0,64,52.0,36,35.0,11,21.0,21,...,55,53.0,27,21.0,54,48.0,21,27.0,63,59.0
8,2017-05-09,32,21.0,73,59.0,31,39.0,18,23.0,18,...,60,59.0,19,24.0,62,54.0,39,30.0,79,66.0
9,2017-05-10,17,22.0,55,60.0,44,39.0,14,23.0,18,...,58,61.0,26,24.0,47,54.0,30,31.0,62,67.0


## updated may 2017 data model and its forecasting 

In [34]:
dict_jun = model_train_forecast(may_month_data, jun_month_data)

14:58:29 - cmdstanpy - INFO - Chain [1] start processing
14:58:29 - cmdstanpy - INFO - Chain [1] done processing
14:58:30 - cmdstanpy - INFO - Chain [1] start processing
14:58:30 - cmdstanpy - INFO - Chain [1] done processing
14:58:31 - cmdstanpy - INFO - Chain [1] start processing
14:58:31 - cmdstanpy - INFO - Chain [1] done processing
14:58:32 - cmdstanpy - INFO - Chain [1] start processing
14:58:32 - cmdstanpy - INFO - Chain [1] done processing
14:58:33 - cmdstanpy - INFO - Chain [1] start processing
14:58:33 - cmdstanpy - INFO - Chain [1] done processing
14:58:34 - cmdstanpy - INFO - Chain [1] start processing
14:58:34 - cmdstanpy - INFO - Chain [1] done processing
14:58:35 - cmdstanpy - INFO - Chain [1] start processing
14:58:35 - cmdstanpy - INFO - Chain [1] done processing
14:58:35 - cmdstanpy - INFO - Chain [1] start processing
14:58:36 - cmdstanpy - INFO - Chain [1] done processing
14:58:36 - cmdstanpy - INFO - Chain [1] start processing
14:58:37 - cmdstanpy - INFO - Chain [1]

14:59:26 - cmdstanpy - INFO - Chain [1] done processing
14:59:26 - cmdstanpy - INFO - Chain [1] start processing
14:59:26 - cmdstanpy - INFO - Chain [1] done processing
14:59:26 - cmdstanpy - INFO - Chain [1] start processing
14:59:26 - cmdstanpy - INFO - Chain [1] done processing
14:59:27 - cmdstanpy - INFO - Chain [1] start processing
14:59:27 - cmdstanpy - INFO - Chain [1] done processing
14:59:27 - cmdstanpy - INFO - Chain [1] start processing
14:59:27 - cmdstanpy - INFO - Chain [1] done processing
14:59:27 - cmdstanpy - INFO - Chain [1] start processing
14:59:28 - cmdstanpy - INFO - Chain [1] done processing
14:59:28 - cmdstanpy - INFO - Chain [1] start processing
14:59:28 - cmdstanpy - INFO - Chain [1] done processing
14:59:28 - cmdstanpy - INFO - Chain [1] start processing
14:59:28 - cmdstanpy - INFO - Chain [1] done processing
14:59:29 - cmdstanpy - INFO - Chain [1] start processing
14:59:29 - cmdstanpy - INFO - Chain [1] done processing
14:59:29 - cmdstanpy - INFO - Chain [1] 

In [35]:
dict_jun["acuracy"]

0.881594741763573

In [36]:
dict_jun["diff_df"]

,sales_1_forecast_diff,sales_2_forecast_diff,sales_3_forecast_diff,sales_4_forecast_diff,sales_5_forecast_diff,sales_6_forecast_diff,sales_7_forecast_diff,sales_8_forecast_diff,sales_9_forecast_diff,sales_10_forecast_diff,...,sales_41_forecast_diff,sales_42_forecast_diff,sales_43_forecast_diff,sales_44_forecast_diff,sales_45_forecast_diff,sales_46_forecast_diff,sales_47_forecast_diff,sales_48_forecast_diff,sales_49_forecast_diff,sales_50_forecast_diff
0,-4.0,-18.0,-13.0,4.0,-4.0,2.0,-20.0,-12.0,6.0,-10.0,...,-2.0,3.0,4.0,-4.0,1.0,11.0,-4.0,1.0,-4.0,-1.0
1,2.0,-17.0,-7.0,8.0,4.0,15.0,-3.0,-10.0,4.0,-3.0,...,-2.0,4.0,-25.0,3.0,-9.0,-9.0,1.0,-1.0,-5.0,-3.0
2,-8.0,-34.0,12.0,-5.0,-3.0,19.0,-11.0,-15.0,-4.0,9.0,...,5.0,-5.0,4.0,-21.0,-25.0,-9.0,-8.0,-2.0,-12.0,4.0
3,3.0,-28.0,-2.0,-6.0,-11.0,2.0,-13.0,-8.0,-9.0,-15.0,...,2.0,-14.0,-11.0,3.0,-20.0,-4.0,-3.0,-3.0,-6.0,-14.0
4,-1.0,0.0,-9.0,-3.0,-2.0,13.0,-3.0,0.0,6.0,19.0,...,1.0,2.0,7.0,1.0,3.0,12.0,6.0,-1.0,4.0,4.0
5,2.0,-6.0,1.0,-3.0,1.0,-1.0,-10.0,-7.0,9.0,-11.0,...,1.0,-17.0,0.0,3.0,-3.0,1.0,-1.0,-3.0,1.0,-6.0
6,-4.0,-5.0,1.0,4.0,0.0,-1.0,-12.0,6.0,-13.0,-26.0,...,-2.0,1.0,-7.0,-1.0,-17.0,-9.0,5.0,-2.0,6.0,-1.0
7,-1.0,-3.0,-9.0,-5.0,-2.0,-5.0,-15.0,-30.0,-7.0,-1.0,...,-2.0,4.0,-6.0,4.0,4.0,-2.0,0.0,-10.0,-5.0,-5.0
8,11.0,-7.0,-2.0,1.0,-1.0,-4.0,-9.0,-22.0,-12.0,15.0,...,0.0,1.0,4.0,-1.0,-25.0,-11.0,-1.0,-13.0,2.0,0.0
9,-12.0,-13.0,4.0,-12.0,-3.0,2.0,-14.0,-7.0,-14.0,6.0,...,-3.0,2.0,-9.0,-16.0,5.0,11.0,1.0,4.0,-5.0,-13.0


In [37]:
dict_jun["true_forecat_df"]

,date,sales_1,sales_1_forecast,sales_2,sales_2_forecast,sales_3,sales_3_forecast,sales_4,sales_4_forecast,sales_5,...,sales_46,sales_46_forecast,sales_47,sales_47_forecast,sales_48,sales_48_forecast,sales_49,sales_49_forecast,sales_50,sales_50_forecast
0,2017-06-01,29,25.0,83,65.0,57,44.0,22,26.0,27,...,57,68.0,30,26.0,58,59.0,38,34.0,78,77.0
1,2017-06-02,25,27.0,85,68.0,54,47.0,19,27.0,19,...,80,71.0,26,27.0,64,63.0,39,34.0,83,80.0
2,2017-06-03,36,28.0,106,72.0,37,49.0,34,29.0,27,...,84,75.0,36,28.0,67,65.0,48,36.0,81,85.0
3,2017-06-04,27,30.0,103,75.0,54,52.0,36,30.0,36,...,83,79.0,32,29.0,72,69.0,45,39.0,102,88.0
4,2017-06-05,22,21.0,54,54.0,48,39.0,24,21.0,21,...,46,58.0,16,22.0,51,50.0,24,28.0,62,66.0
5,2017-06-06,22,24.0,67,61.0,41,42.0,27,24.0,20,...,63,64.0,26,25.0,60,57.0,30,31.0,80,74.0
6,2017-06-07,29,25.0,67,62.0,42,43.0,20,24.0,21,...,74,65.0,20,25.0,58,56.0,26,32.0,76,75.0
7,2017-06-08,26,25.0,68,65.0,53,44.0,31,26.0,24,...,71,69.0,26,26.0,69,59.0,39,34.0,84,79.0
8,2017-06-09,16,27.0,75,68.0,49,47.0,25,26.0,24,...,83,72.0,28,27.0,76,63.0,33,35.0,82,82.0
9,2017-06-10,41,29.0,85,72.0,44,48.0,40,28.0,27,...,64,75.0,27,28.0,61,65.0,42,37.0,100,87.0


## updated jun 2017 data model and its forecasting 

In [38]:
dict_jul = model_train_forecast(jun_month_data, jul_month_data)

14:59:53 - cmdstanpy - INFO - Chain [1] start processing
14:59:53 - cmdstanpy - INFO - Chain [1] done processing
14:59:54 - cmdstanpy - INFO - Chain [1] start processing
14:59:54 - cmdstanpy - INFO - Chain [1] done processing
14:59:55 - cmdstanpy - INFO - Chain [1] start processing
14:59:55 - cmdstanpy - INFO - Chain [1] done processing
14:59:56 - cmdstanpy - INFO - Chain [1] start processing
14:59:56 - cmdstanpy - INFO - Chain [1] done processing
14:59:57 - cmdstanpy - INFO - Chain [1] start processing
14:59:57 - cmdstanpy - INFO - Chain [1] done processing
14:59:58 - cmdstanpy - INFO - Chain [1] start processing
14:59:58 - cmdstanpy - INFO - Chain [1] done processing
14:59:59 - cmdstanpy - INFO - Chain [1] start processing
14:59:59 - cmdstanpy - INFO - Chain [1] done processing
15:00:00 - cmdstanpy - INFO - Chain [1] start processing
15:00:00 - cmdstanpy - INFO - Chain [1] done processing
15:00:01 - cmdstanpy - INFO - Chain [1] start processing
15:00:01 - cmdstanpy - INFO - Chain [1]

15:00:51 - cmdstanpy - INFO - Chain [1] done processing
15:00:52 - cmdstanpy - INFO - Chain [1] start processing
15:00:52 - cmdstanpy - INFO - Chain [1] done processing
15:00:52 - cmdstanpy - INFO - Chain [1] start processing
15:00:52 - cmdstanpy - INFO - Chain [1] done processing
15:00:52 - cmdstanpy - INFO - Chain [1] start processing
15:00:52 - cmdstanpy - INFO - Chain [1] done processing
15:00:53 - cmdstanpy - INFO - Chain [1] start processing
15:00:53 - cmdstanpy - INFO - Chain [1] done processing
15:00:53 - cmdstanpy - INFO - Chain [1] start processing
15:00:53 - cmdstanpy - INFO - Chain [1] done processing
15:00:53 - cmdstanpy - INFO - Chain [1] start processing
15:00:54 - cmdstanpy - INFO - Chain [1] done processing
15:00:54 - cmdstanpy - INFO - Chain [1] start processing
15:00:54 - cmdstanpy - INFO - Chain [1] done processing
15:00:54 - cmdstanpy - INFO - Chain [1] start processing
15:00:54 - cmdstanpy - INFO - Chain [1] done processing
15:00:55 - cmdstanpy - INFO - Chain [1] 

In [39]:
dict_jul["acuracy"]

0.8811516916272348

In [40]:
dict_jul["diff_df"]

,sales_1_forecast_diff,sales_2_forecast_diff,sales_3_forecast_diff,sales_4_forecast_diff,sales_5_forecast_diff,sales_6_forecast_diff,sales_7_forecast_diff,sales_8_forecast_diff,sales_9_forecast_diff,sales_10_forecast_diff,...,sales_41_forecast_diff,sales_42_forecast_diff,sales_43_forecast_diff,sales_44_forecast_diff,sales_45_forecast_diff,sales_46_forecast_diff,sales_47_forecast_diff,sales_48_forecast_diff,sales_49_forecast_diff,sales_50_forecast_diff
0,0.0,-5.0,6.0,-7.0,-2.0,-5.0,-10.0,-8.0,-17.0,-6.0,...,-10.0,-2.0,3.0,-3.0,-24.0,-14.0,2.0,-6.0,9.0,-5.0
1,-6.0,-17.0,-8.0,-8.0,0.0,-8.0,-10.0,2.0,-8.0,-28.0,...,-4.0,-5.0,-9.0,-12.0,-12.0,-18.0,9.0,-15.0,-3.0,-9.0
2,6.0,8.0,2.0,5.0,1.0,-13.0,0.0,4.0,-12.0,-4.0,...,6.0,6.0,5.0,-8.0,-6.0,5.0,2.0,7.0,7.0,-2.0
3,4.0,1.0,-6.0,-4.0,-1.0,-1.0,-20.0,2.0,11.0,-12.0,...,-7.0,3.0,-18.0,-12.0,-1.0,2.0,-2.0,10.0,-7.0,-10.0
4,4.0,1.0,1.0,-4.0,0.0,1.0,13.0,16.0,-7.0,-2.0,...,0.0,-10.0,-4.0,0.0,7.0,9.0,-5.0,2.0,-5.0,-16.0
5,-3.0,4.0,1.0,2.0,-3.0,-12.0,-15.0,-9.0,-8.0,6.0,...,-8.0,9.0,-10.0,2.0,-6.0,-9.0,-8.0,-4.0,6.0,-9.0
6,-3.0,-5.0,-2.0,-7.0,3.0,-26.0,-14.0,-6.0,-12.0,0.0,...,4.0,-3.0,-31.0,0.0,2.0,-18.0,3.0,12.0,-2.0,10.0
7,4.0,-15.0,-8.0,-4.0,0.0,-14.0,-9.0,-2.0,-8.0,-9.0,...,5.0,4.0,-16.0,1.0,-23.0,-7.0,3.0,-12.0,2.0,-2.0
8,0.0,-15.0,-7.0,-7.0,3.0,-9.0,-15.0,-29.0,-3.0,-12.0,...,3.0,-12.0,-6.0,-13.0,-14.0,-16.0,-3.0,-5.0,8.0,-2.0
9,8.0,13.0,-4.0,7.0,2.0,2.0,2.0,14.0,9.0,-2.0,...,1.0,1.0,8.0,-3.0,-6.0,15.0,-7.0,7.0,3.0,1.0


In [41]:
dict_jul["true_forecat_df"]

,date,sales_1,sales_1_forecast,sales_2,sales_2_forecast,sales_3,sales_3_forecast,sales_4,sales_4_forecast,sales_5,...,sales_46,sales_46_forecast,sales_47,sales_47_forecast,sales_48,sales_48_forecast,sales_49,sales_49_forecast,sales_50,sales_50_forecast
0,2017-07-01,31,31.0,83,78.0,41,47.0,37,30.0,26,...,91,77.0,29,31.0,76,70.0,31,40.0,91,86.0
1,2017-07-02,38,32.0,98,81.0,58,50.0,39,31.0,26,...,99,81.0,23,32.0,90,75.0,46,43.0,97,88.0
2,2017-07-03,18,24.0,52,60.0,36,38.0,18,23.0,18,...,55,60.0,23,25.0,49,56.0,25,32.0,68,66.0
3,2017-07-04,23,27.0,66,67.0,47,41.0,30,26.0,23,...,65,67.0,29,27.0,53,63.0,42,35.0,84,74.0
4,2017-07-05,23,27.0,67,68.0,41,42.0,30,26.0,22,...,59,68.0,32,27.0,61,63.0,40,35.0,91,75.0
5,2017-07-06,31,28.0,67,71.0,43,44.0,26,28.0,26,...,81,72.0,37,29.0,70,66.0,31,37.0,88,79.0
6,2017-07-07,32,29.0,80,75.0,49,47.0,35,28.0,21,...,93,75.0,27,30.0,58,70.0,40,38.0,72,82.0
7,2017-07-08,27,31.0,94,79.0,57,49.0,35,31.0,25,...,86,79.0,28,31.0,84,72.0,38,40.0,89,87.0
8,2017-07-09,32,32.0,97,82.0,59,52.0,39,32.0,23,...,99,83.0,36,33.0,82,77.0,34,42.0,92,90.0
9,2017-07-10,16,24.0,49,62.0,43,39.0,17,24.0,18,...,47,62.0,32,25.0,51,58.0,29,32.0,67,68.0


## updated jul 2017 data model and its forecasting 

In [42]:
dict_aug = model_train_forecast(jul_month_data, aug_month_data)

15:01:20 - cmdstanpy - INFO - Chain [1] start processing
15:01:20 - cmdstanpy - INFO - Chain [1] done processing
15:01:21 - cmdstanpy - INFO - Chain [1] start processing
15:01:21 - cmdstanpy - INFO - Chain [1] done processing
15:01:22 - cmdstanpy - INFO - Chain [1] start processing
15:01:22 - cmdstanpy - INFO - Chain [1] done processing
15:01:23 - cmdstanpy - INFO - Chain [1] start processing
15:01:23 - cmdstanpy - INFO - Chain [1] done processing
15:01:24 - cmdstanpy - INFO - Chain [1] start processing
15:01:24 - cmdstanpy - INFO - Chain [1] done processing
15:01:25 - cmdstanpy - INFO - Chain [1] start processing
15:01:25 - cmdstanpy - INFO - Chain [1] done processing
15:01:26 - cmdstanpy - INFO - Chain [1] start processing
15:01:26 - cmdstanpy - INFO - Chain [1] done processing
15:01:27 - cmdstanpy - INFO - Chain [1] start processing
15:01:27 - cmdstanpy - INFO - Chain [1] done processing
15:01:28 - cmdstanpy - INFO - Chain [1] start processing
15:01:28 - cmdstanpy - INFO - Chain [1]

15:02:21 - cmdstanpy - INFO - Chain [1] done processing
15:02:21 - cmdstanpy - INFO - Chain [1] start processing
15:02:22 - cmdstanpy - INFO - Chain [1] done processing
15:02:22 - cmdstanpy - INFO - Chain [1] start processing
15:02:22 - cmdstanpy - INFO - Chain [1] done processing
15:02:22 - cmdstanpy - INFO - Chain [1] start processing
15:02:22 - cmdstanpy - INFO - Chain [1] done processing
15:02:23 - cmdstanpy - INFO - Chain [1] start processing
15:02:23 - cmdstanpy - INFO - Chain [1] done processing
15:02:23 - cmdstanpy - INFO - Chain [1] start processing
15:02:23 - cmdstanpy - INFO - Chain [1] done processing
15:02:24 - cmdstanpy - INFO - Chain [1] start processing
15:02:24 - cmdstanpy - INFO - Chain [1] done processing
15:02:24 - cmdstanpy - INFO - Chain [1] start processing
15:02:24 - cmdstanpy - INFO - Chain [1] done processing
15:02:24 - cmdstanpy - INFO - Chain [1] start processing
15:02:25 - cmdstanpy - INFO - Chain [1] done processing
15:02:25 - cmdstanpy - INFO - Chain [1] 

In [43]:
dict_aug["acuracy"]

0.8829869856841128

In [44]:
dict_aug["diff_df"]

,sales_1_forecast_diff,sales_2_forecast_diff,sales_3_forecast_diff,sales_4_forecast_diff,sales_5_forecast_diff,sales_6_forecast_diff,sales_7_forecast_diff,sales_8_forecast_diff,sales_9_forecast_diff,sales_10_forecast_diff,...,sales_41_forecast_diff,sales_42_forecast_diff,sales_43_forecast_diff,sales_44_forecast_diff,sales_45_forecast_diff,sales_46_forecast_diff,sales_47_forecast_diff,sales_48_forecast_diff,sales_49_forecast_diff,sales_50_forecast_diff
0,7.0,10.0,-1.0,10.0,3.0,9.0,18.0,11.0,5.0,5.0,...,8.0,13.0,-21.0,-3.0,11.0,-10.0,3.0,-8.0,8.0,8.0
1,6.0,9.0,11.0,5.0,8.0,22.0,-8.0,-9.0,10.0,-7.0,...,10.0,-2.0,7.0,-5.0,27.0,6.0,1.0,11.0,12.0,1.0
2,1.0,-1.0,6.0,3.0,1.0,-3.0,8.0,16.0,5.0,-8.0,...,0.0,10.0,1.0,1.0,2.0,17.0,2.0,3.0,6.0,18.0
3,1.0,-4.0,17.0,3.0,2.0,-7.0,-3.0,7.0,-11.0,-8.0,...,-11.0,10.0,6.0,-1.0,-20.0,-3.0,-6.0,8.0,-1.0,-3.0
4,-6.0,12.0,-21.0,3.0,6.0,0.0,4.0,1.0,11.0,-19.0,...,2.0,3.0,3.0,-10.0,4.0,0.0,-2.0,-4.0,7.0,-19.0
5,0.0,-9.0,4.0,7.0,4.0,-17.0,1.0,-5.0,4.0,-4.0,...,10.0,-8.0,8.0,-3.0,-10.0,7.0,-4.0,-16.0,-3.0,-5.0
6,1.0,4.0,0.0,-1.0,4.0,-1.0,2.0,1.0,3.0,5.0,...,5.0,2.0,9.0,0.0,12.0,-1.0,1.0,8.0,5.0,-3.0
7,-5.0,-17.0,4.0,1.0,5.0,11.0,-3.0,-2.0,-4.0,6.0,...,-1.0,2.0,-2.0,10.0,-6.0,-14.0,3.0,-5.0,11.0,9.0
8,-4.0,7.0,14.0,-2.0,1.0,-3.0,10.0,2.0,-9.0,15.0,...,-7.0,0.0,-9.0,-5.0,-6.0,2.0,-5.0,8.0,10.0,11.0
9,-7.0,-5.0,-1.0,5.0,4.0,5.0,2.0,7.0,0.0,10.0,...,-2.0,0.0,-10.0,-9.0,3.0,0.0,-11.0,-1.0,5.0,2.0


In [45]:
dict_aug["true_forecat_df"]

,date,sales_1,sales_1_forecast,sales_2,sales_2_forecast,sales_3,sales_3_forecast,sales_4,sales_4_forecast,sales_5,...,sales_46,sales_46_forecast,sales_47,sales_47_forecast,sales_48,sales_48_forecast,sales_49,sales_49_forecast,sales_50,sales_50_forecast
0,2017-08-01,18,25.0,58,68.0,43,42.0,16,26.0,18,...,75,65.0,22,25.0,67,59.0,27,35.0,68,76.0
1,2017-08-02,19,25.0,59,68.0,31,42.0,20,25.0,13,...,60,66.0,24,25.0,47,58.0,23,35.0,75,76.0
2,2017-08-03,25,26.0,72,71.0,37,43.0,24,27.0,22,...,52,69.0,24,26.0,58,61.0,31,37.0,61,79.0
3,2017-08-04,26,27.0,77,73.0,29,46.0,25,28.0,21,...,75,72.0,33,27.0,56,64.0,39,38.0,85,82.0
4,2017-08-05,35,29.0,65,77.0,68,47.0,26,29.0,18,...,74,74.0,30,28.0,70,66.0,32,39.0,105,86.0
5,2017-08-06,29,29.0,89,80.0,46,50.0,23,30.0,21,...,71,78.0,33,29.0,86,70.0,45,42.0,93,88.0
6,2017-08-07,20,21.0,54,58.0,36,36.0,23,22.0,15,...,56,55.0,21,22.0,42,50.0,26,31.0,67,64.0
7,2017-08-08,28,23.0,81,64.0,35,39.0,24,25.0,16,...,75,61.0,21,24.0,61,56.0,23,34.0,63,72.0
8,2017-08-09,28,24.0,58,65.0,26,40.0,26,24.0,20,...,60,62.0,29,24.0,48,56.0,24,34.0,61,72.0
9,2017-08-10,31,24.0,72,67.0,42,41.0,21,26.0,19,...,65,65.0,37,26.0,59,58.0,31,36.0,73,75.0


## updated aug 2017 data model and its forecasting 

In [46]:
dict_sep = model_train_forecast(aug_month_data, sep_month_data)

15:02:50 - cmdstanpy - INFO - Chain [1] start processing
15:02:50 - cmdstanpy - INFO - Chain [1] done processing
15:02:51 - cmdstanpy - INFO - Chain [1] start processing
15:02:51 - cmdstanpy - INFO - Chain [1] done processing
15:02:52 - cmdstanpy - INFO - Chain [1] start processing
15:02:52 - cmdstanpy - INFO - Chain [1] done processing
15:02:53 - cmdstanpy - INFO - Chain [1] start processing
15:02:53 - cmdstanpy - INFO - Chain [1] done processing
15:02:54 - cmdstanpy - INFO - Chain [1] start processing
15:02:54 - cmdstanpy - INFO - Chain [1] done processing
15:02:55 - cmdstanpy - INFO - Chain [1] start processing
15:02:55 - cmdstanpy - INFO - Chain [1] done processing
15:02:56 - cmdstanpy - INFO - Chain [1] start processing
15:02:57 - cmdstanpy - INFO - Chain [1] done processing
15:02:57 - cmdstanpy - INFO - Chain [1] start processing
15:02:58 - cmdstanpy - INFO - Chain [1] done processing
15:02:59 - cmdstanpy - INFO - Chain [1] start processing
15:02:59 - cmdstanpy - INFO - Chain [1]

15:03:52 - cmdstanpy - INFO - Chain [1] done processing
15:03:53 - cmdstanpy - INFO - Chain [1] start processing
15:03:53 - cmdstanpy - INFO - Chain [1] done processing
15:03:53 - cmdstanpy - INFO - Chain [1] start processing
15:03:53 - cmdstanpy - INFO - Chain [1] done processing
15:03:53 - cmdstanpy - INFO - Chain [1] start processing
15:03:54 - cmdstanpy - INFO - Chain [1] done processing
15:03:54 - cmdstanpy - INFO - Chain [1] start processing
15:03:54 - cmdstanpy - INFO - Chain [1] done processing
15:03:54 - cmdstanpy - INFO - Chain [1] start processing
15:03:54 - cmdstanpy - INFO - Chain [1] done processing
15:03:55 - cmdstanpy - INFO - Chain [1] start processing
15:03:55 - cmdstanpy - INFO - Chain [1] done processing
15:03:55 - cmdstanpy - INFO - Chain [1] start processing
15:03:55 - cmdstanpy - INFO - Chain [1] done processing
15:03:56 - cmdstanpy - INFO - Chain [1] start processing
15:03:56 - cmdstanpy - INFO - Chain [1] done processing
15:03:56 - cmdstanpy - INFO - Chain [1] 

In [47]:
dict_sep["acuracy"]

0.8841042017827355

In [48]:
dict_sep["diff_df"]

,sales_1_forecast_diff,sales_2_forecast_diff,sales_3_forecast_diff,sales_4_forecast_diff,sales_5_forecast_diff,sales_6_forecast_diff,sales_7_forecast_diff,sales_8_forecast_diff,sales_9_forecast_diff,sales_10_forecast_diff,...,sales_41_forecast_diff,sales_42_forecast_diff,sales_43_forecast_diff,sales_44_forecast_diff,sales_45_forecast_diff,sales_46_forecast_diff,sales_47_forecast_diff,sales_48_forecast_diff,sales_49_forecast_diff,sales_50_forecast_diff
0,-12.0,3.0,3.0,6.0,-4.0,11.0,-1.0,2.0,7.0,25.0,...,9.0,-7.0,14.0,2.0,4.0,2.0,-4.0,-1.0,5.0,12.0
1,2.0,-5.0,-1.0,6.0,-2.0,5.0,-11.0,7.0,6.0,8.0,...,-3.0,-14.0,2.0,4.0,3.0,3.0,11.0,8.0,9.0,-9.0
2,6.0,-11.0,8.0,9.0,1.0,8.0,6.0,16.0,-4.0,18.0,...,2.0,-7.0,4.0,9.0,18.0,10.0,-3.0,10.0,2.0,10.0
3,8.0,3.0,-5.0,-3.0,0.0,-9.0,-2.0,7.0,-1.0,16.0,...,-2.0,-2.0,-1.0,-1.0,19.0,-3.0,12.0,-4.0,-4.0,7.0
4,-3.0,7.0,-5.0,3.0,12.0,10.0,0.0,-6.0,-7.0,6.0,...,10.0,1.0,9.0,4.0,0.0,6.0,4.0,13.0,-3.0,13.0
5,0.0,-8.0,-12.0,2.0,0.0,8.0,8.0,12.0,-12.0,9.0,...,-1.0,-6.0,-2.0,0.0,10.0,3.0,-3.0,4.0,-8.0,10.0
6,-5.0,5.0,-4.0,5.0,-5.0,16.0,4.0,-7.0,6.0,16.0,...,7.0,5.0,-1.0,1.0,11.0,5.0,1.0,1.0,-8.0,4.0
7,-3.0,2.0,-3.0,6.0,9.0,7.0,8.0,-14.0,0.0,-14.0,...,-4.0,-12.0,1.0,6.0,0.0,3.0,3.0,-9.0,-6.0,-13.0
8,-1.0,-1.0,9.0,8.0,-4.0,3.0,9.0,-11.0,1.0,-11.0,...,-3.0,-2.0,-12.0,5.0,3.0,5.0,3.0,-2.0,-17.0,13.0
9,-2.0,9.0,12.0,-2.0,0.0,2.0,-14.0,8.0,-8.0,-13.0,...,-1.0,-2.0,-1.0,3.0,-1.0,-16.0,6.0,-11.0,0.0,0.0


In [49]:
dict_sep["true_forecat_df"]

,date,sales_1,sales_1_forecast,sales_2,sales_2_forecast,sales_3,sales_3_forecast,sales_4,sales_4_forecast,sales_5,...,sales_46,sales_46_forecast,sales_47,sales_47_forecast,sales_48,sales_48_forecast,sales_49,sales_49_forecast,sales_50,sales_50_forecast
0,2017-09-01,38,26.0,67,70.0,41,44.0,20,26.0,26,...,67,69.0,32,28.0,63,62.0,29,34.0,64,76.0
1,2017-09-02,26,28.0,78,73.0,47,46.0,22,28.0,25,...,68,71.0,18,29.0,56,64.0,27,36.0,90,81.0
2,2017-09-03,23,29.0,87,76.0,41,49.0,20,29.0,23,...,66,76.0,33,30.0,58,68.0,36,38.0,73,83.0
3,2017-09-04,12,20.0,51,54.0,40,35.0,24,21.0,17,...,57,54.0,10,22.0,52,48.0,31,27.0,54,61.0
4,2017-09-05,26,23.0,54,61.0,43,38.0,20,23.0,7,...,54,60.0,21,25.0,42,55.0,33,30.0,55,68.0
5,2017-09-06,24,24.0,69,61.0,51,39.0,21,23.0,19,...,57,60.0,28,25.0,50,54.0,37,29.0,59,69.0
6,2017-09-07,29,24.0,59,64.0,44,40.0,20,25.0,25,...,59,64.0,25,26.0,56,57.0,40,32.0,68,72.0
7,2017-09-08,28,25.0,65,67.0,46,43.0,20,26.0,12,...,64,67.0,23,26.0,69,60.0,39,33.0,88,75.0
8,2017-09-09,28,27.0,71,70.0,36,45.0,20,28.0,26,...,65,70.0,24,27.0,64,62.0,51,34.0,67,80.0
9,2017-09-10,30,28.0,64,73.0,36,48.0,31,29.0,23,...,90,74.0,22,28.0,77,66.0,37,37.0,82,82.0


## updated sep 2017 data model and its forecasting 

In [50]:
dict_oct = model_train_forecast(sep_month_data, oct_month_data)

15:04:23 - cmdstanpy - INFO - Chain [1] start processing
15:04:23 - cmdstanpy - INFO - Chain [1] done processing
15:04:24 - cmdstanpy - INFO - Chain [1] start processing
15:04:24 - cmdstanpy - INFO - Chain [1] done processing
15:04:25 - cmdstanpy - INFO - Chain [1] start processing
15:04:25 - cmdstanpy - INFO - Chain [1] done processing
15:04:26 - cmdstanpy - INFO - Chain [1] start processing
15:04:26 - cmdstanpy - INFO - Chain [1] done processing
15:04:27 - cmdstanpy - INFO - Chain [1] start processing
15:04:27 - cmdstanpy - INFO - Chain [1] done processing
15:04:28 - cmdstanpy - INFO - Chain [1] start processing
15:04:28 - cmdstanpy - INFO - Chain [1] done processing
15:04:29 - cmdstanpy - INFO - Chain [1] start processing
15:04:29 - cmdstanpy - INFO - Chain [1] done processing
15:04:30 - cmdstanpy - INFO - Chain [1] start processing
15:04:30 - cmdstanpy - INFO - Chain [1] done processing
15:04:31 - cmdstanpy - INFO - Chain [1] start processing
15:04:31 - cmdstanpy - INFO - Chain [1]

15:05:26 - cmdstanpy - INFO - Chain [1] done processing
15:05:26 - cmdstanpy - INFO - Chain [1] start processing
15:05:26 - cmdstanpy - INFO - Chain [1] done processing
15:05:26 - cmdstanpy - INFO - Chain [1] start processing
15:05:26 - cmdstanpy - INFO - Chain [1] done processing
15:05:27 - cmdstanpy - INFO - Chain [1] start processing
15:05:27 - cmdstanpy - INFO - Chain [1] done processing
15:05:27 - cmdstanpy - INFO - Chain [1] start processing
15:05:27 - cmdstanpy - INFO - Chain [1] done processing
15:05:28 - cmdstanpy - INFO - Chain [1] start processing
15:05:28 - cmdstanpy - INFO - Chain [1] done processing
15:05:28 - cmdstanpy - INFO - Chain [1] start processing
15:05:28 - cmdstanpy - INFO - Chain [1] done processing
15:05:29 - cmdstanpy - INFO - Chain [1] start processing
15:05:29 - cmdstanpy - INFO - Chain [1] done processing
15:05:29 - cmdstanpy - INFO - Chain [1] start processing
15:05:29 - cmdstanpy - INFO - Chain [1] done processing
15:05:30 - cmdstanpy - INFO - Chain [1] 

In [51]:
dict_oct["acuracy"]

0.8751828723229669

In [52]:
dict_oct["diff_df"]

,sales_1_forecast_diff,sales_2_forecast_diff,sales_3_forecast_diff,sales_4_forecast_diff,sales_5_forecast_diff,sales_6_forecast_diff,sales_7_forecast_diff,sales_8_forecast_diff,sales_9_forecast_diff,sales_10_forecast_diff,...,sales_41_forecast_diff,sales_42_forecast_diff,sales_43_forecast_diff,sales_44_forecast_diff,sales_45_forecast_diff,sales_46_forecast_diff,sales_47_forecast_diff,sales_48_forecast_diff,sales_49_forecast_diff,sales_50_forecast_diff
0,6.0,14.0,8.0,4.0,2.0,1.0,3.0,-3.0,0.0,4.0,...,0.0,-1.0,-2.0,9.0,5.0,5.0,-4.0,2.0,1.0,14.0
1,6.0,-4.0,0.0,-4.0,1.0,9.0,7.0,5.0,6.0,-9.0,...,-1.0,10.0,5.0,9.0,7.0,13.0,3.0,1.0,4.0,10.0
2,3.0,4.0,8.0,3.0,-4.0,-2.0,7.0,22.0,-1.0,-11.0,...,3.0,0.0,-6.0,4.0,8.0,1.0,10.0,-9.0,2.0,10.0
3,7.0,5.0,2.0,-4.0,3.0,-4.0,4.0,18.0,12.0,11.0,...,9.0,4.0,9.0,4.0,13.0,0.0,-1.0,13.0,-5.0,-2.0
4,3.0,4.0,-3.0,-3.0,1.0,7.0,16.0,14.0,13.0,17.0,...,-4.0,-8.0,8.0,-6.0,19.0,-2.0,3.0,0.0,-5.0,18.0
5,5.0,13.0,6.0,-1.0,-2.0,12.0,19.0,19.0,6.0,5.0,...,0.0,0.0,-1.0,0.0,-4.0,20.0,-4.0,3.0,0.0,-2.0
6,4.0,10.0,8.0,5.0,2.0,-5.0,-2.0,-7.0,1.0,18.0,...,6.0,-8.0,10.0,-18.0,9.0,4.0,-8.0,6.0,8.0,18.0
7,8.0,14.0,-2.0,4.0,2.0,-3.0,-5.0,-7.0,-3.0,5.0,...,-1.0,12.0,-1.0,-6.0,-2.0,7.0,7.0,2.0,-9.0,-8.0
8,9.0,5.0,13.0,-4.0,1.0,2.0,10.0,23.0,4.0,17.0,...,-3.0,-1.0,9.0,1.0,6.0,-6.0,0.0,0.0,4.0,-6.0
9,-2.0,7.0,-1.0,-2.0,3.0,11.0,10.0,13.0,8.0,7.0,...,5.0,-4.0,-18.0,2.0,-8.0,9.0,1.0,7.0,0.0,-2.0


In [53]:
dict_oct["true_forecat_df"]

,date,sales_1,sales_1_forecast,sales_2,sales_2_forecast,sales_3,sales_3_forecast,sales_4,sales_4_forecast,sales_5,...,sales_46,sales_46_forecast,sales_47,sales_47_forecast,sales_48,sales_48_forecast,sales_49,sales_49_forecast,sales_50,sales_50_forecast
0,2017-10-01,21,27.0,60,74.0,38,46.0,22,26.0,20,...,69,74.0,31,27.0,62,64.0,35,36.0,66,80.0
1,2017-10-02,12,18.0,56,52.0,32,32.0,21,17.0,15,...,39,52.0,16,19.0,44,45.0,21,25.0,47,57.0
2,2017-10-03,18,21.0,55,59.0,28,36.0,17,20.0,22,...,57,58.0,12,22.0,60,51.0,26,28.0,55,65.0
3,2017-10-04,15,22.0,55,60.0,34,36.0,24,20.0,15,...,58,58.0,23,22.0,37,50.0,33,28.0,67,65.0
4,2017-10-05,20,23.0,59,63.0,40,37.0,25,22.0,19,...,64,62.0,20,23.0,54,54.0,35,30.0,50,68.0
5,2017-10-06,19,24.0,54,67.0,34,40.0,23,22.0,22,...,45,65.0,28,24.0,54,57.0,31,31.0,74,72.0
6,2017-10-07,22,26.0,61,71.0,34,42.0,19,24.0,19,...,64,68.0,33,25.0,53,59.0,25,33.0,58,76.0
7,2017-10-08,19,27.0,60,74.0,47,45.0,21,25.0,20,...,65,72.0,20,27.0,61,63.0,45,36.0,86,78.0
8,2017-10-09,9,18.0,46,51.0,18,31.0,21,17.0,14,...,56,50.0,19,19.0,44,44.0,21,25.0,61,55.0
9,2017-10-10,23,21.0,52,59.0,35,34.0,22,20.0,14,...,47,56.0,20,21.0,43,50.0,28,28.0,65,63.0


## updated oct 2017 data model and its forecasting 

In [54]:
dict_nov = model_train_forecast(oct_month_data, nov_month_data)

15:05:57 - cmdstanpy - INFO - Chain [1] start processing
15:05:57 - cmdstanpy - INFO - Chain [1] done processing
15:05:58 - cmdstanpy - INFO - Chain [1] start processing
15:05:58 - cmdstanpy - INFO - Chain [1] done processing
15:05:59 - cmdstanpy - INFO - Chain [1] start processing
15:05:59 - cmdstanpy - INFO - Chain [1] done processing
15:06:00 - cmdstanpy - INFO - Chain [1] start processing
15:06:00 - cmdstanpy - INFO - Chain [1] done processing
15:06:01 - cmdstanpy - INFO - Chain [1] start processing
15:06:01 - cmdstanpy - INFO - Chain [1] done processing
15:06:02 - cmdstanpy - INFO - Chain [1] start processing
15:06:02 - cmdstanpy - INFO - Chain [1] done processing
15:06:03 - cmdstanpy - INFO - Chain [1] start processing
15:06:03 - cmdstanpy - INFO - Chain [1] done processing
15:06:04 - cmdstanpy - INFO - Chain [1] start processing
15:06:04 - cmdstanpy - INFO - Chain [1] done processing
15:06:05 - cmdstanpy - INFO - Chain [1] start processing
15:06:05 - cmdstanpy - INFO - Chain [1]

15:07:00 - cmdstanpy - INFO - Chain [1] done processing
15:07:00 - cmdstanpy - INFO - Chain [1] start processing
15:07:01 - cmdstanpy - INFO - Chain [1] done processing
15:07:01 - cmdstanpy - INFO - Chain [1] start processing
15:07:01 - cmdstanpy - INFO - Chain [1] done processing
15:07:01 - cmdstanpy - INFO - Chain [1] start processing
15:07:02 - cmdstanpy - INFO - Chain [1] done processing
15:07:02 - cmdstanpy - INFO - Chain [1] start processing
15:07:02 - cmdstanpy - INFO - Chain [1] done processing
15:07:02 - cmdstanpy - INFO - Chain [1] start processing
15:07:02 - cmdstanpy - INFO - Chain [1] done processing
15:07:03 - cmdstanpy - INFO - Chain [1] start processing
15:07:03 - cmdstanpy - INFO - Chain [1] done processing
15:07:03 - cmdstanpy - INFO - Chain [1] start processing
15:07:03 - cmdstanpy - INFO - Chain [1] done processing
15:07:04 - cmdstanpy - INFO - Chain [1] start processing
15:07:04 - cmdstanpy - INFO - Chain [1] done processing
15:07:04 - cmdstanpy - INFO - Chain [1] 

In [55]:
dict_nov["acuracy"]

0.8741140480484292

In [56]:
dict_nov["diff_df"]

,sales_1_forecast_diff,sales_2_forecast_diff,sales_3_forecast_diff,sales_4_forecast_diff,sales_5_forecast_diff,sales_6_forecast_diff,sales_7_forecast_diff,sales_8_forecast_diff,sales_9_forecast_diff,sales_10_forecast_diff,...,sales_41_forecast_diff,sales_42_forecast_diff,sales_43_forecast_diff,sales_44_forecast_diff,sales_45_forecast_diff,sales_46_forecast_diff,sales_47_forecast_diff,sales_48_forecast_diff,sales_49_forecast_diff,sales_50_forecast_diff
0,6.0,-6.0,-15.0,-3.0,7.0,-7.0,10.0,-3.0,-13.0,-8.0,...,7.0,-14.0,-10.0,15.0,6.0,5.0,4.0,-2.0,7.0,5.0
1,2.0,-6.0,-4.0,1.0,1.0,3.0,9.0,2.0,13.0,1.0,...,4.0,-10.0,-4.0,-8.0,2.0,-1.0,1.0,-8.0,-5.0,9.0
2,6.0,-10.0,-2.0,3.0,5.0,-1.0,-15.0,-6.0,-12.0,-5.0,...,9.0,2.0,-13.0,5.0,2.0,-1.0,-13.0,4.0,-1.0,-9.0
3,-8.0,2.0,2.0,8.0,0.0,-3.0,12.0,4.0,0.0,-3.0,...,0.0,-1.0,3.0,-3.0,20.0,-3.0,0.0,-8.0,-4.0,1.0
4,4.0,0.0,-5.0,5.0,-6.0,6.0,-6.0,-10.0,0.0,-12.0,...,3.0,-2.0,3.0,-2.0,4.0,-11.0,3.0,-7.0,2.0,-2.0
5,2.0,-17.0,-10.0,-2.0,1.0,-25.0,-2.0,6.0,-6.0,3.0,...,-8.0,5.0,6.0,1.0,-15.0,-5.0,3.0,0.0,8.0,2.0
6,-1.0,-8.0,0.0,-4.0,1.0,-12.0,1.0,-16.0,2.0,25.0,...,5.0,-1.0,-4.0,-1.0,3.0,0.0,9.0,-10.0,-6.0,15.0
7,9.0,3.0,6.0,5.0,0.0,-4.0,-6.0,-9.0,7.0,-5.0,...,4.0,3.0,9.0,-14.0,-4.0,6.0,-2.0,10.0,8.0,9.0
8,3.0,2.0,2.0,1.0,4.0,-1.0,4.0,2.0,14.0,-21.0,...,8.0,-1.0,3.0,-8.0,6.0,8.0,-5.0,-4.0,-4.0,2.0
9,6.0,10.0,20.0,-7.0,5.0,-1.0,11.0,1.0,-1.0,-7.0,...,-13.0,0.0,-9.0,2.0,-1.0,2.0,4.0,-7.0,-5.0,7.0


In [57]:
dict_nov["true_forecat_df"]

,date,sales_1,sales_1_forecast,sales_2,sales_2_forecast,sales_3,sales_3_forecast,sales_4,sales_4_forecast,sales_5,...,sales_46,sales_46_forecast,sales_47,sales_47_forecast,sales_48,sales_48_forecast,sales_49,sales_49_forecast,sales_50,sales_50_forecast
0,2017-11-01,16,22.0,63,57.0,50,35.0,23,20.0,11,...,53,58.0,18,22.0,51,49.0,21,28.0,59,64.0
1,2017-11-02,21,23.0,67,61.0,41,37.0,22,23.0,18,...,63,62.0,23,24.0,61,53.0,35,30.0,59,68.0
2,2017-11-03,18,24.0,75,65.0,43,41.0,20,23.0,15,...,67,66.0,38,25.0,53,57.0,32,31.0,82,73.0
3,2017-11-04,34,26.0,68,70.0,41,43.0,17,25.0,21,...,73,70.0,27,27.0,68,60.0,37,33.0,77,78.0
4,2017-11-05,23,27.0,74,74.0,51,46.0,21,26.0,28,...,86,75.0,25,28.0,71,64.0,34,36.0,83,81.0
5,2017-11-06,17,19.0,69,52.0,43,33.0,20,18.0,15,...,58,53.0,18,21.0,45,45.0,17,25.0,56,58.0
6,2017-11-07,23,22.0,68,60.0,37,37.0,25,21.0,17,...,60,60.0,15,24.0,62,52.0,34,28.0,52,67.0
7,2017-11-08,14,23.0,58,61.0,31,37.0,16,21.0,18,...,55,61.0,26,24.0,42,52.0,21,29.0,58,67.0
8,2017-11-09,21,24.0,62,64.0,37,39.0,22,23.0,16,...,57,65.0,30,25.0,60,56.0,36,32.0,69,71.0
9,2017-11-10,19,25.0,58,68.0,23,43.0,31,24.0,15,...,67,69.0,23,27.0,66,59.0,38,33.0,68,75.0


## updated nov 2017 data model and its forecasting 

In [58]:
dict_des = model_train_forecast(nov_month_data, des_month_data)

15:07:31 - cmdstanpy - INFO - Chain [1] start processing
15:07:31 - cmdstanpy - INFO - Chain [1] done processing
15:07:32 - cmdstanpy - INFO - Chain [1] start processing
15:07:32 - cmdstanpy - INFO - Chain [1] done processing
15:07:33 - cmdstanpy - INFO - Chain [1] start processing
15:07:33 - cmdstanpy - INFO - Chain [1] done processing
15:07:34 - cmdstanpy - INFO - Chain [1] start processing
15:07:34 - cmdstanpy - INFO - Chain [1] done processing
15:07:35 - cmdstanpy - INFO - Chain [1] start processing
15:07:35 - cmdstanpy - INFO - Chain [1] done processing
15:07:36 - cmdstanpy - INFO - Chain [1] start processing
15:07:36 - cmdstanpy - INFO - Chain [1] done processing
15:07:37 - cmdstanpy - INFO - Chain [1] start processing
15:07:37 - cmdstanpy - INFO - Chain [1] done processing
15:07:38 - cmdstanpy - INFO - Chain [1] start processing
15:07:38 - cmdstanpy - INFO - Chain [1] done processing
15:07:39 - cmdstanpy - INFO - Chain [1] start processing
15:07:39 - cmdstanpy - INFO - Chain [1]

15:08:36 - cmdstanpy - INFO - Chain [1] done processing
15:08:37 - cmdstanpy - INFO - Chain [1] start processing
15:08:37 - cmdstanpy - INFO - Chain [1] done processing
15:08:37 - cmdstanpy - INFO - Chain [1] start processing
15:08:37 - cmdstanpy - INFO - Chain [1] done processing
15:08:38 - cmdstanpy - INFO - Chain [1] start processing
15:08:38 - cmdstanpy - INFO - Chain [1] done processing
15:08:38 - cmdstanpy - INFO - Chain [1] start processing
15:08:38 - cmdstanpy - INFO - Chain [1] done processing
15:08:39 - cmdstanpy - INFO - Chain [1] start processing
15:08:39 - cmdstanpy - INFO - Chain [1] done processing
15:08:39 - cmdstanpy - INFO - Chain [1] start processing
15:08:39 - cmdstanpy - INFO - Chain [1] done processing
15:08:40 - cmdstanpy - INFO - Chain [1] start processing
15:08:40 - cmdstanpy - INFO - Chain [1] done processing
15:08:40 - cmdstanpy - INFO - Chain [1] start processing
15:08:40 - cmdstanpy - INFO - Chain [1] done processing
15:08:40 - cmdstanpy - INFO - Chain [1] 

In [59]:
dict_des["acuracy"]

0.8488921273092168

In [60]:
dict_des["diff_df"]

,sales_1_forecast_diff,sales_2_forecast_diff,sales_3_forecast_diff,sales_4_forecast_diff,sales_5_forecast_diff,sales_6_forecast_diff,sales_7_forecast_diff,sales_8_forecast_diff,sales_9_forecast_diff,sales_10_forecast_diff,...,sales_41_forecast_diff,sales_42_forecast_diff,sales_43_forecast_diff,sales_44_forecast_diff,sales_45_forecast_diff,sales_46_forecast_diff,sales_47_forecast_diff,sales_48_forecast_diff,sales_49_forecast_diff,sales_50_forecast_diff
0,2.0,2.0,7.0,-1.0,6.0,0.0,3.0,9.0,-1.0,4.0,...,5.0,10.0,10.0,7.0,10.0,23.0,1.0,12.0,12.0,14.0
1,7.0,-2.0,7.0,8.0,2.0,12.0,22.0,30.0,10.0,17.0,...,-8.0,12.0,17.0,11.0,32.0,6.0,5.0,13.0,3.0,9.0
2,-8.0,5.0,6.0,9.0,2.0,1.0,17.0,21.0,15.0,10.0,...,7.0,9.0,3.0,16.0,19.0,-3.0,-1.0,23.0,10.0,12.0
3,7.0,9.0,6.0,6.0,1.0,-8.0,0.0,-3.0,10.0,18.0,...,-3.0,7.0,2.0,13.0,12.0,7.0,1.0,10.0,-2.0,16.0
4,-3.0,3.0,15.0,3.0,0.0,5.0,12.0,23.0,12.0,10.0,...,1.0,13.0,10.0,5.0,-2.0,8.0,-1.0,6.0,3.0,6.0
5,0.0,-2.0,-2.0,5.0,1.0,16.0,4.0,6.0,15.0,12.0,...,3.0,3.0,10.0,-6.0,9.0,2.0,2.0,0.0,5.0,19.0
6,0.0,4.0,-3.0,6.0,7.0,11.0,7.0,7.0,-2.0,-1.0,...,-1.0,8.0,14.0,1.0,5.0,4.0,1.0,11.0,4.0,13.0
7,2.0,10.0,6.0,4.0,11.0,-8.0,-3.0,7.0,-5.0,-10.0,...,2.0,4.0,7.0,-3.0,0.0,10.0,7.0,4.0,-1.0,7.0
8,-7.0,7.0,-1.0,0.0,4.0,-2.0,-2.0,14.0,17.0,27.0,...,2.0,7.0,8.0,0.0,5.0,-1.0,8.0,2.0,-2.0,8.0
9,1.0,10.0,-7.0,4.0,1.0,-11.0,9.0,11.0,2.0,9.0,...,7.0,9.0,2.0,4.0,12.0,12.0,3.0,6.0,8.0,3.0


In [61]:
dict_des["true_forecat_df"]

,date,sales_1,sales_1_forecast,sales_2,sales_2_forecast,sales_3,sales_3_forecast,sales_4,sales_4_forecast,sales_5,...,sales_46,sales_46_forecast,sales_47,sales_47_forecast,sales_48,sales_48_forecast,sales_49,sales_49_forecast,sales_50,sales_50_forecast
0,2017-12-01,19,21.0,56,58.0,30,37.0,23,22.0,13,...,35,58.0,21,22.0,38,50.0,17,29.0,51,65.0
1,2017-12-02,16,23.0,64,62.0,31,38.0,16,24.0,18,...,54,60.0,18,23.0,39,52.0,27,30.0,60,69.0
2,2017-12-03,31,23.0,59,64.0,34,40.0,15,24.0,19,...,67,64.0,25,24.0,33,56.0,22,32.0,59,71.0
3,2017-12-04,7,14.0,33,42.0,21,27.0,10,16.0,13,...,34,41.0,15,16.0,25,35.0,22,20.0,31,47.0
4,2017-12-05,20,17.0,45,48.0,15,30.0,15,18.0,16,...,39,47.0,19,18.0,35,41.0,20,23.0,48,54.0
5,2017-12-06,17,17.0,50,48.0,32,30.0,13,18.0,15,...,45,47.0,16,18.0,40,40.0,18,23.0,35,54.0
6,2017-12-07,18,18.0,47,51.0,34,31.0,14,20.0,10,...,46,50.0,18,19.0,32,43.0,21,25.0,44,57.0
7,2017-12-08,17,19.0,44,54.0,27,33.0,16,20.0,6,...,43,53.0,13,20.0,42,46.0,27,26.0,53,60.0
8,2017-12-09,27,20.0,50,57.0,36,35.0,22,22.0,14,...,56,55.0,13,21.0,46,48.0,29,27.0,56,64.0
9,2017-12-10,20,21.0,50,60.0,44,37.0,18,22.0,18,...,47,59.0,19,22.0,46,52.0,22,30.0,63,66.0


In [1]:
######################## rough work #####################

In [7]:
import pandas as pd
a = pd.DataFrame({"a":[1,2], "b":[4, 5]})
a

,a,b
0,1,4
1,2,5


In [6]:
a.empty

False

In [14]:
["g"] + list(a.columns)

['g', 'a', 'b']

In [17]:
["g"] + [list(a.columns)][0]

['g', 'a', 'b']

In [19]:
["g"] + [i for i in list(a.columns)]

['g', 'a', 'b']

In [ ]:
a.set_index("date")

In [23]:
if a :
    print("s")

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().